## Functions and Data

In [1]:
import datetime
import copy


import numpy as np
from numpy.random import normal as rnorm, multinomial as rmultinomial
import scipy.stats
from scipy.special import logsumexp, loggamma
from sklearn.mixture import GaussianMixture
from sklearn.metrics import r2_score, mean_squared_error, roc_auc_score
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge, LogisticRegression, LogisticRegressionCV
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
from sklearn.model_selection import KFold

import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm


plt.style.use(['seaborn-talk'])

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


In [2]:
import warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=sm.tools.sm_exceptions.DomainWarning)

In [3]:
data_W = pd.read_excel(r"data/Hourly Weather and No Outlier Wachapreague Data.xlsx", parse_dates=[r"Date"])
data_W = data_W.drop(
    columns=["Wind Direction (degrees)", "Water Level Anomalies (m)"]
    ).rename(
    columns = {"Date": "time_min", 
                "Wind Speed (m/s)": "wind",
               "Air Pressure (mb)": "airpressure",
               "Precipitation (mm/hr)": "precipitation", 
               "Temperature ©": "temperature",
               "Salinity (ppt)": "salinity",
               "DO (mg/L)": "ODO",
               "Water Level (m)": "waterlevel",
               "Log10(Chl+1) (log10(ug/L))": "log10_chlorophyll",
              }
    ).set_index("time_min")

data_W["chlorophyll"] = data_W["log10_chlorophyll"].map(lambda x: np.power(10, x))
data_W["date"] = data_W.index.date
#data_W = data_W.dropna()

# data_W = pd.read_excel(r"data/Corrected W All.xlsx", sheet_name = "Sheet1", parse_dates=[r"Combine"],)
# data_W = data_W.drop(
#     columns=["MM/DD/YY", "HH:mm:SS", "pH (mv)", "ODO (%sat)", "BGA-PE (ug/L)", "Battery (volts)", "Sonde SN", "Unnamed: 15"]
#     ).rename(
#     columns = {"Combine": "time_min",
#                "Temp ('C)": "temperature",
#                "SpCond (ms/cm)": "conductivity",
#                "Salinity (ppt)": "salinity",
#                "ODO (mg/L)": "ODO",
#                "Turb (NTU)": "turbidity",
#                "Chl (ug/L)": "chlorophyll",
#               }
#     ).set_index("time_min")
# data_W["date"] = data_W.index.date
# data_W["log10_chlorophyll"] = data_W["chlorophyll"].map(np.log10)
# data_W = data_W.dropna()


In [4]:
data_W

,wind,airpressure,precipitation,temperature,salinity,ODO,log10_chlorophyll,waterlevel,chlorophyll,date
time_min,,,,,,,,,,
2016-03-25 20:00:00,1.870,1012.210000,0.76,16.70975,30.9550,8.0425,0.360495,0.938,2.293480,2016-03-25
2016-03-25 21:00:00,2.500,1013.730000,0.53,16.43975,31.2575,7.8200,0.319070,1.127,2.084826,2016-03-25
2016-03-25 22:00:00,3.690,1014.980000,0.20,15.83275,31.4425,7.8650,0.327645,1.268,2.126401,2016-03-25
2016-03-25 23:00:00,4.180,1015.490000,0.03,15.43950,31.4775,7.8525,0.330899,1.324,2.142390,2016-03-25
2016-03-26 00:00:00,3.150,1016.750000,0.00,15.45900,31.3925,7.7400,0.325592,1.238,2.116372,2016-03-26
...,...,...,...,...,...,...,...,...,...,...
2022-12-31 18:00:00,0.625,1011.325000,0.02,7.93175,30.2575,10.6000,0.358752,0.8,2.284292,2022-12-31
2022-12-31 19:00:00,1.800,1011.310000,NaN,8.10550,30.1100,10.5450,0.359835,NaN,2.289997,2022-12-31
2022-12-31 20:00:00,2.120,1010.420000,NaN,8.65100,28.9200,10.2250,0.429598,NaN,2.689042,2022-12-31


In [5]:

data_WW = pd.read_excel(r"data/VIMS WQ Data.xlsx", sheet_name = "WW", skiprows = 7, na_values = {"pH": [0.0]}, parse_dates=[r"Date/Time Combined", r"MM/DD/YY"], )
data_WW_2 = pd.read_excel(r"data/2022-Willis_Wharf ALL Raw (1).xlsx", skiprows = 7, na_values = {"pH": [0.0]}, parse_dates=[r"Date/Time Combined", r"MM/DD/YY"], )

data_WW = data_WW.drop(
    columns=["Unnamed: 17", "Unnamed: 18", "MM/DD/YY.1", "Flagged or Deleted Data/Notes", "Log10 chl", 
             "pH (mv)", "ODO (%sat)", "BGA-PE (ug/L)", "Battery (volts)", "Sonde SN", "HH:mm:SS", "TSS (mg/L)", "Day"],
    ).rename(
    columns = {"MM/DD/YY": "date", 
               "Date/Time Combined": "time_min",
               "Temp ('C)": "temperature",
               "SpCond (ms/cm)": "conductivity",
               "Salinity (ppt)": "salinity",
               "ODO (mg/L)": "ODO",
               "Turb (NTU)": "turbidity",
               "Chl (ug/L)": "chlorophyll",
              }
    ).set_index("time_min")

data_WW_2 = data_WW_2.drop(
    columns=["Unnamed: 16", "Unnamed: 17", "Unnamed: 18", "Unnamed: 19", "Unnamed: 20", 
             "pH (mv)", "ODO (%sat)", "BGA-PE (ug/L)", "Battery (volts)", "Sonde SN", "HH:mm:SS", "TSS (mg/L)"],
    ).rename(
    columns = {"MM/DD/YY": "date", 
               "Date/Time Combined": "time_min",
               "Temp ('C)": "temperature",
               "SpCond (ms/cm)": "conductivity",
               "Salinity (ppt)": "salinity",
               "ODO (mg/L)": "ODO",
               "Turb (NTU)": "turbidity",
               "Chl (ug/L)": "chlorophyll",
              }
    ).set_index("time_min")

data_WW = pd.concat([data_WW[data_WW.index.year < 2022], data_WW_2])
data_WW["log10_chlorophyll"] = data_WW["chlorophyll"].map(np.log10)
#data_WW = data_WW.dropna()


d:\Tools\anaconda3\lib\site-packages\pandas\core\base.py:880: RuntimeWarning: divide by zero encountered in log10
  new_values = map_f(values, mapper)
d:\Tools\anaconda3\lib\site-packages\pandas\core\base.py:880: RuntimeWarning: invalid value encountered in log10
  new_values = map_f(values, mapper)


In [6]:
data_WW

,date,temperature,conductivity,salinity,pH,ODO,turbidity,chlorophyll,log10_chlorophyll
time_min,,,,,,,,,
2018-10-12 12:53:04,2018-10-12,24.221,46.978,30.55,7.81,5.84,21.72,3.87,0.587711
2018-10-12 14:03:38,2018-10-12,24.615,46.755,30.38,7.86,6.37,19.17,6.34,0.802089
2018-10-12 14:17:40,2018-10-12,24.757,46.513,30.20,7.85,6.54,20.72,6.97,0.843233
2018-10-12 14:32:41,2018-10-12,24.898,46.381,30.10,7.85,6.71,21.37,7.67,0.884795
2018-10-12 14:47:41,2018-10-12,24.940,46.315,30.05,7.86,6.81,23.71,7.33,0.865104
...,...,...,...,...,...,...,...,...,...
2022-12-24 05:02:12,2022-12-24,4.105,46.071,29.17,7.84,9.80,17.23,3.00,0.477121
2022-12-24 05:17:12,2022-12-24,3.908,46.023,29.11,7.84,9.79,17.85,3.09,0.489958
2022-12-24 05:32:12,2022-12-24,3.921,46.073,29.15,7.83,9.72,21.78,3.06,0.485721


In [7]:
def matmul_log(A, log_b):
    res = []
    for tmp_line in A:
        if np.any(tmp_line == 0.) :
            if np.all(tmp_line == 0.) :
                res.append(logsumexp(np.log([1 / len(tmp_line) for i in range(len(tmp_line))]) + log_b))
            else:
                res.append(logsumexp(np.log(tmp_line[tmp_line != 0]) + log_b[tmp_line != 0]))
        else:
            res.append(logsumexp(np.log(tmp_line) + log_b))
    return np.array(res)


In [8]:
class MSLR():
    
    def __init__(self, n_components = 2, covariance_type="full", n_iter = 10, reg_method = "OLS", kargs_reg = None):
        self.n_components = n_components
        self.n_iter = n_iter
        self.covariance_type = covariance_type 
        self.tol = 0.0
        
        if kargs_reg is None:
            kargs_reg = dict()
        self.kargs_reg = kargs_reg
            
        if reg_method in {"OLS", "LR", "LinearRegression", }:
            self.reg_method = LinearRegression
        elif reg_method in {"Ridge", }:
            self.reg_method = Ridge
        elif reg_method in {"RidgeCV", }:
            self.reg_method = RidgeCV
        elif reg_method in {"LinearSVR", "LinearSVM", }:
            self.reg_method = LinearSVR
        else:
            self.reg_method = reg_method
        
        return
    
    
    def fit_predict(self, X, Y, X_test, is_multiple_sequence = False, forecast_horizon = 1):
        
        #matmul_log = lambda A, log_b: np.array([logsumexp(np.log(tmp_line) + log_b) for tmp_line in A])
        
        if is_multiple_sequence:
            list_X, list_Y = [np.array(i) for i in X], [np.array(i) for i in Y]
            list_X_test = [np.array(i) for i in X_test] if X_test is not None else None
        else:
            list_X, list_Y = [np.array(X)], [np.array(Y)]
            list_X_test = [np.array(X_test)] if X_test is not None else None
        
        marginal_X, marginal_Y = np.concatenate(list_X), np.concatenate(list_Y)
        gmm = GaussianMixture(n_components = self.n_components, covariance_type = self.covariance_type, random_state=434)
        gmm.fit(marginal_Y)
        
        n_seq = len(list_X)
        list_T = [len(i) for i in list_X]
        
        p_X, p = len(list_X[0][0]), len(list_Y[0][0])
        K = self.n_components
        self.n_features = p
        
        self.transmat_ = np.array([[1. / K for j in range(K)] for i in range(K)])

        self.list_loglik_ = []
        for epoch in range(self.n_iter):

            # M-step
            if epoch != 0:
                tmp_weight_mat = []
                for cur_log_forward_prob, cur_log_backward_prob in zip(list_cur_log_forward_prob, list_cur_log_backward_prob):
                    tmp_log_weight_mat = cur_log_forward_prob + cur_log_backward_prob
                    tmp_log_weight_mat -= logsumexp(tmp_log_weight_mat[-1, :])
                    tmp_weight_mat.append(np.exp(tmp_log_weight_mat))
                tmp_weight_mat = np.concatenate(tmp_weight_mat)
            else:
                tmp_weight_mat = gmm.predict_proba(marginal_Y)
                
            list_lr_cov = []
            for ii in range(K):
                tmp_weight = tmp_weight_mat[:, ii]
                tmp_lr = self.reg_method(**self.kargs_reg)
                tmp_lr.fit(marginal_X, marginal_Y, sample_weight = tmp_weight)
                tmp_resid = marginal_Y - tmp_lr.predict(marginal_X)
                if self.covariance_type == "full":
                    tmp_cov_sum = np.zeros(shape = (p, p))
                    for jj in range(len(marginal_X)):
                        tmp_x = tmp_resid[jj, :]
                        tmp_cov_sum += np.outer(tmp_x, tmp_x) * tmp_weight[jj]
                    tmp_cov = tmp_cov_sum / np.sum(tmp_weight) 
                elif self.covariance_type == "diag":
                    tmp_cov = np.array([max(np.sum(tmp_weight * (tmp_resid[:, j] ** 2)) / np.sum(tmp_weight), 1e-12) for j in range(p)])
                list_lr_cov.append((tmp_lr, tmp_cov))
            self.list_lr_cov = list_lr_cov
            
            list_cur_mat_log_b = self._calc_emission_mat(list_X, list_Y)
            list_list_log_emission = [[np.copy(list_cur_mat_log_b[index_X][i, :]) for i in range(T)] for index_X, T in enumerate(list_T)]
                        
            if epoch != 0:
                cur_log_initprob = [list_cur_log_forward_prob[i][0] + list_cur_log_backward_prob[i][0] for i in range(n_seq)]
            else:
                cur_log_initprob = [[np.log(i) if i != 0 else -100 for i in line] for line in gmm.predict_proba([Y[0] for Y in list_Y])]
            
            cur_log_initprob = np.sum(cur_log_initprob, axis = 0)
            cur_log_initprob -= logsumexp(cur_log_initprob)
            self.startprob_ = np.exp(cur_log_initprob)
            self.log_startprob_ = cur_log_initprob

            # E-step      
            list_cur_log_forward_prob = [[] for i in range(n_seq)]
            for index_X, X in enumerate(list_X):
                for tt in range(len(X)):
                    if tt == 0:
                        tmp_log_prob = cur_log_initprob + list_cur_mat_log_b[index_X][0, :]
                    else:
                        tmp_log_prob = matmul_log(np.transpose(self.transmat_), list_cur_log_forward_prob[index_X][-1]) + list_cur_mat_log_b[index_X][tt, :]
                    list_cur_log_forward_prob[index_X].append(tmp_log_prob)
            list_cur_log_forward_prob = [np.array(i) for i in list_cur_log_forward_prob]
            
            list_cur_log_backward_prob = [[0 for i in range(T)] for T in list_T]
            for index_X, (X, T) in enumerate(zip(list_X, list_T)):
                for tt in range(T - 1, -1, -1):
                    if tt == T - 1:
                        tmp_log_prob = np.array([np.log(1.) for i in range(K)])
                    else:
                        tmp_log_prob = []
                        for ii in range(K):
                            tmp_log_prob.append(logsumexp([np.log(self.transmat_[ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
                        tmp_log_prob = np.array(tmp_log_prob)
                    list_cur_log_backward_prob[index_X][tt] = tmp_log_prob
            list_cur_log_backward_prob = [np.array(i) for i in list_cur_log_backward_prob]
            
            list_tmp_array2d_log_gamma = [i + j for (i, j) in zip(list_cur_log_forward_prob, list_cur_log_backward_prob)]
            list_tmp_array1d_log_gamma_sum = [[logsumexp(i[t, :]) for t in range(len(i))] for i in list_tmp_array2d_log_gamma]
            
            list_tmp_array3d_log_epsilon = [np.zeros(shape = (T - 1, K, K)) for T in list_T]
            for index_X, (X, T) in enumerate(zip(list_X, list_T)):
                for tt in range(T - 1):
                    for ii in range(K):
                        for jj in range(K):
                            list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(self.transmat_[ii, jj]) + \
                                    list_cur_log_backward_prob[index_X][tt + 1, jj] + list_cur_mat_log_b[index_X][tt + 1, jj] - list_tmp_array1d_log_gamma_sum[index_X][tt]            
        
            self.transmat_ = self._calc_transmat(list_tmp_array3d_log_epsilon)
            
            loglik = np.sum([logsumexp(i[-1]) for i in list_cur_log_forward_prob])
            self.list_loglik_.append(loglik)
            
            if (len(self.list_loglik_) >= 2) and (abs(self.list_loglik_[-1] - self.list_loglik_[-2]) < self.tol):
                break
            
            #print("epoch % s, loglik = % s" % (epoch, loglik))
            #print(cur_list_mvn_mean_cov)
            #print(cur_transmat)
        
        self.list_last_posterior = [i[-1] + j[-1] for (i, j) in zip(list_cur_log_backward_prob, list_cur_log_forward_prob)]
        for ii in range(n_seq):
            tmp = self.list_last_posterior[ii]
            tmp -= np.max(tmp)
            tmp = np.exp(tmp)
            tmp /= np.sum(tmp)
            self.list_last_posterior[ii] = tmp
        
        if list_X_test is None:
            pred = None
        else:
            pred = []
            for ii in range(n_seq):
                tmp_pred_prob = self.list_last_posterior[ii] @ np.linalg.matrix_power(self.transmat_, forecast_horizon)
                tmp_pred = 0
                for tmp_k in range(K):
                    tmp_pred += list_lr_cov[tmp_k][0].predict([list_X_test[ii]])[0] * tmp_pred_prob[tmp_k]
                pred.append(tmp_pred)

            if not is_multiple_sequence:
                pred = pred[0]
        
        return pred
    
    
    def fit(self, X, Y, is_multiple_sequence = False):
        self.fit_predict(X = X, Y = Y, X_test = None, is_multiple_sequence = is_multiple_sequence)
        return
    
    
    def _calc_transmat(self, list_tmp_array3d_log_epsilon):
        K = self.n_components
        cur_transmat = np.zeros((K, K)) 
        for ii in range(K):
            tmp_list_log_prob = []
            for jj in range(K):
                tmp_log_prob = []
                for index_X in range(len(list_tmp_array3d_log_epsilon)):
                    tmp_log_prob.append(logsumexp(list_tmp_array3d_log_epsilon[index_X][:, ii, jj]))
                tmp_list_log_prob.append(logsumexp(tmp_log_prob))
            tmp_list_log_prob = np.array(tmp_list_log_prob)
            tmp_list_log_prob -= np.max(tmp_list_log_prob)
            tmp_prob = np.exp(tmp_list_log_prob)
            tmp_prob /= sum(tmp_prob)
            cur_transmat[ii, :] = tmp_prob
        return cur_transmat
    
    
    def _calc_emission_mat(self, list_X, list_Y):
        
        list_T = [len(i) for i in list_X]
        p, K = self.n_features, self.n_components
        
        list_cur_mat_log_b = [np.zeros(shape = (i, K)) for i in list_T]
        for ii, (tmp_lr, tmp_cov) in enumerate(self.list_lr_cov):
            for index_X, (X, Y) in enumerate(zip(list_X, list_Y)):
                tmp_resid = Y - tmp_lr.predict(X)
                if self.covariance_type == "full":
                    list_cur_mat_log_b[index_X][:, ii] = scipy.stats.multivariate_normal.logpdf(tmp_resid, mean=[0] * p, cov=tmp_cov)
                elif self.covariance_type == "diag":
                    tmp_logprob = 0
                    for jj in range(p):
                        tmp_logprob += scipy.stats.norm.logpdf(tmp_resid[:, jj], loc = 0, scale = np.sqrt(tmp_cov[jj]))
                    list_cur_mat_log_b[index_X][:, ii] = tmp_logprob

        return list_cur_mat_log_b
    
    
# def _test_MSLR():
    
#     list_beta = [np.array([[0.5, 0.5], [0.5, 0.5], [0.5, 0.5]]), 
#                  np.array([[0.1, 0.5], [0.8, 0.5], [0.2, 0.6]]), 
#                  np.array([[-0.5, 0.7], [-0.5, -0.8], [-0.5, -0.2]]),]
#     list_intercept = [np.array([1., 2.]), np.array([8., 2.]), np.array([5., 3.])]
#     list_cov = [np.array([[1., 0.1], [0.1, 1.]]), np.array([[1.5, 0.1], [0.1, 1.5]]), np.array([[3., 0.1], [0.1, 3.]])]
#     transmat = np.array([[0.9, 0.05,  0.05], [ 0.3, 0.6,  0.1], [ 0.3,  0.2, 0.5]])
#     init_prob = np.array([1 / 3, 1 / 2, 1 / 6])
#     T = 10000
    
#     list_X, list_Y = [], []
#     list_X_test = []
#     for ii in range(5):
        
#         X = np.random.normal(loc=1.0, scale=5.0, size=(T, 3))
#         X_test = np.random.normal(loc=1.0, scale=5.0, size=(1, 3))
#         Y, list_h_label = [], []
#         for tt in range(T):
#             if tt == 0:
#                 tmp_h_label = np.argmax(np.random.multinomial(n = 1, pvals = init_prob, size = 1))
#             else:
#                 tmp_h_label = np.argmax(np.random.multinomial(n = 1, pvals = transmat[tmp_h_label], size = 1))
#             list_h_label.append(tmp_h_label)
#             tmp_Y = X[tt, :] @ list_beta[tmp_h_label] + list_intercept[tmp_h_label] + np.random.multivariate_normal(mean = [0, 0], cov = list_cov[tmp_h_label])
#             Y.append(tmp_Y)
#         Y = np.array(Y)
        
#         list_X.append(X)
#         list_Y.append(Y)
#         list_X_test.append(X_test)
        
#     mslr = MSLR(n_components=3, covariance_type="full", n_iter=10)
#     pred = mslr.fit_predict(list_X, list_Y, list_X_test, is_multiple_sequence=True)
#     print(pred)
#     print(mslr.transmat_)
#     print(mslr.startprob_)
#     for tmp_lr, tmp_cov in mslr.list_lr_cov:
#         print(tmp_lr.coef_)
#         print(tmp_lr.intercept_)
#         print(tmp_cov)

#     return


# _test_MSLR()


In [9]:
class MSLRX():
    
    def __init__(self, n_components = 2, covariance_type="full", n_iter = 10, reg_method = "OLS", kargs_reg = None, 
                 is_logistic_regression_CV = False, logistic_regression_C = 1e10, is_logistic_regression_standardized = False,
                is_state_coef_indep = False, is_ordinal_logit = False, tol_loglik = 0.):
        
        self.n_components = n_components
        self.covariance_type = covariance_type 
        self.n_iter = n_iter
        self.min_iter = 3
        self.is_logistic_regression_CV = is_logistic_regression_CV
        self.logistic_regression_C = logistic_regression_C
        self.is_state_coef_indep = is_state_coef_indep
        self.is_ordinal_logit = is_ordinal_logit    # only valid when the response is 1-dim so we could sort it
        self.tol_loglik = tol_loglik
        self.is_print_loglik = False
        
        if kargs_reg is None:
            kargs_reg = dict()
        self.kargs_reg = kargs_reg
            
        if reg_method in {"OLS", "LR", "LinearRegression", }:
            self.reg_method = LinearRegression
        elif reg_method in {"Ridge", }:
            self.reg_method = Ridge
        elif reg_method in {"RidgeCV", }:
            self.reg_method = RidgeCV
        elif reg_method in {"LinearSVR", "LinearSVM", }:
            self.reg_method = LinearSVR
        else:
            self.reg_method = reg_method
        
        self.is_logistic_regression_standardized = is_logistic_regression_standardized
        if is_logistic_regression_standardized:
            self.standardizer = StandardScaler()
            
        return
    
    
    def _est_trans_mat(self, list_exog):
        
        list_list_trans_mat = []
        for exog in list_exog:
            T, K = len(exog), self.n_components
            list_trans_mat = []
            for tmp_exog in exog:
                tmp_trans_mat = []
                for ii in range(K):
                    if self.is_state_coef_indep or self.is_ordinal_logit:
                        if self.is_initialized:
                            
                            ii_index = self.list_sorted_index[ii] if self.is_ordinal_logit else ii
                            tmp_new_exog = [int(i == ii_index) for i in range(K)]
                            tmp_new_exog.extend(tmp_exog)
                            tmp_prob = self.logit_clf.predict_proba(np.array([tmp_new_exog]))[0]
                            if self.is_ordinal_logit:
                                tmp_prob = tmp_prob[self.list_sorted_index_to_original_index]
                            
                        else:
                            tmp_prob = [1 / K for i in range(K)]
                    else:
                        tmp_log_prob = tmp_exog @ self.list_coef_[ii] + self.list_intercept_[ii]
                        tmp_log_prob -= np.max(tmp_log_prob)
                        tmp_prob = np.exp(tmp_log_prob)
                        tmp_prob /= np.sum(tmp_prob)
                    tmp_trans_mat.append(tmp_prob)
                list_trans_mat.append(tmp_trans_mat)
            list_list_trans_mat.append(np.array(list_trans_mat))
            
        return list_list_trans_mat
    
    
    def _update_coef(self, list_array3d_log_epsilon, list_exog):
        
        if self.is_ordinal_logit:
            self.list_sorted_index = np.argsort([i[0] for i, j in self.list_mvn_mean_cov])
            self.list_sorted_index_to_original_index = [i for i, j in sorted(list(enumerate(self.list_sorted_index)), key = lambda x: x[1])]
            #print(self.list_mvn_mean_cov)
            #print([i[0] for i, j in self.list_mvn_mean_cov])
            #print(self.list_sorted_index, self.list_sorted_index_to_original_index)
        
        if self.is_state_coef_indep or self.is_ordinal_logit:
            K = self.n_components
            if self.is_ordinal_logit:
                logit_clf = mord.LogisticAT(alpha = 1 / self.logistic_regression_C, verbose = 0, max_iter = 100)
            else:
                if self.is_logistic_regression_CV:
                    logit_clf = LogisticRegressionCV(multi_class = "multinomial", max_iter = 100)
                else:
                    logit_clf = LogisticRegression(C = self.logistic_regression_C, multi_class = "multinomial", max_iter = 100)
                
            tmp_X, tmp_y, tmp_weight = [], [], []
            tmp_X_nan, tmp_y_nan, tmp_weight_nan = [], [], []
            
            for ii in range(K):
                for array3d_log_epsilon, exog in zip(list_array3d_log_epsilon, list_exog):
                    T = len(array3d_log_epsilon)
                    for tt in range(T):
                        for jj in range(K):
                            tmp_weight_cell = np.exp(array3d_log_epsilon[tt, ii, jj])
                            
                            ii_index = self.list_sorted_index[ii] if self.is_ordinal_logit else ii
                            jj_index = self.list_sorted_index[jj] if self.is_ordinal_logit else jj
                            
                            tmp_exog = [int(i == ii_index) for i in range(K)]
                            tmp_exog.extend(exog[tt])
                            
                            if np.isnan(tmp_weight_cell) or np.isinf(tmp_weight_cell):
                                tmp_X_nan.append(tmp_exog)
                                tmp_y_nan.append(jj_index)
                                tmp_weight_nan.append(tmp_weight_cell)
                            else:
                                tmp_X.append(tmp_exog)
                                tmp_y.append(jj_index)
                                tmp_weight.append(tmp_weight_cell)
            
            if len(tmp_y_nan) >= 1: 
                print("Warning: encountering % s nan in weight in ghmm_exog._update_coef" % len(tmp_y_nan))
                if len(set(tmp_y)) < K:
                    tmp_X.extend(tmp_X_nan)
                    tmp_y.extend(tmp_y_nan)
                    tmp_weight.extend([1 / K for i in range(len(tmp_weight_nan))])

            logit_clf.fit(np.array(tmp_X), np.array(tmp_y), np.array(tmp_weight)) 
            self.logit_clf = logit_clf
            
        else:
            K = self.n_components
            self.list_coef_, self.list_intercept_ = [], []
            for ii in range(K):
                if self.is_logistic_regression_CV:
                    logit_clf = LogisticRegressionCV(multi_class = "multinomial", max_iter = 100)
                else:
                    logit_clf = LogisticRegression(C = self.logistic_regression_C, multi_class = "multinomial", max_iter = 100)
                tmp_X, tmp_y, tmp_weight = [], [], []
                tmp_X_nan, tmp_y_nan, tmp_weight_nan = [], [], []
                for array3d_log_epsilon, exog in zip(list_array3d_log_epsilon, list_exog):
                    T = len(array3d_log_epsilon)
                    for tt in range(T):
                        for jj in range(K):
                            tmp_weight_cell = np.exp(array3d_log_epsilon[tt, ii, jj])
                            if np.isnan(tmp_weight_cell) or np.isinf(tmp_weight_cell):
                                tmp_X_nan.append(exog[tt])
                                tmp_y_nan.append(jj)
                                tmp_weight_nan.append(tmp_weight_cell)
                            else:
                                tmp_X.append(exog[tt])
                                tmp_y.append(jj)
                                tmp_weight.append(tmp_weight_cell)

                if len(tmp_y_nan) >= 1: 
                    print("Warning: encountering % s nan in weight in ghmm_exog._update_coef" % len(tmp_y_nan))
                    if len(set(tmp_y)) < K:
                        tmp_X, tmp_y, tmp_weight = np.array(tmp_X_nan), np.array(tmp_y_nan), np.array(tmp_weight_nan)
                        tmp_weight = 1

                logit_clf.fit(tmp_X, tmp_y, tmp_weight)

                if self.n_components == 2:
                    tmp_coef = np.array([- logit_clf.coef_.flatten(), logit_clf.coef_.flatten()])
                    tmp_intercept = np.array([- logit_clf.intercept_[0], logit_clf.intercept_[0]])
                else:
                    tmp_coef = logit_clf.coef_[np.argsort(logit_clf.classes_), :]
                    tmp_intercept = logit_clf.intercept_[np.argsort(logit_clf.classes_)]
                self.list_coef_.append(np.transpose(tmp_coef))
                self.list_intercept_.append(tmp_intercept)
                
        return
    
    
    def fit_predict(self, X, Y, exog, X_test, is_multiple_sequence = False, exog_additional = None):
        
        #matmul_log = lambda A, log_b: np.array([logsumexp(np.log(tmp_line) + log_b) for tmp_line in A])
        
        self.is_fitted = True
        self.is_initialized = False
        
        if is_multiple_sequence:
            list_X, list_Y = [np.array(i) for i in X], [np.array(i) for i in Y]
            list_X_test = [np.array(i) for i in X_test] if X_test is not None else None
            list_exog = [np.array(i) for i in exog]
            list_exog_additional = [np.array(i) for i in exog_additional] if exog_additional is not None else None
        else:
            list_X, list_Y = [np.array(X)], [np.array(Y)]
            list_X_test = [np.array(X_test)] if X_test is not None else None
            list_exog = [np.array(exog)]
            list_exog_additional = [np.array(exog_additional)] if exog_additional is not None else None
            
        if self.is_logistic_regression_standardized:
            self.standardizer.fit(np.concatenate(list_exog))
            list_exog = [self.standardizer.transform(i, copy=True) for i in list_exog]
        
        n_seq = len(list_Y)
        list_T = [len(i) for i in list_Y]
        
        p_X, p = len(list_X[0][0]), len(list_Y[0][0])
        p_exog = len(list_exog[0][0])
        K = self.n_components
        self.n_features = p
        
        marginal_X, marginal_Y = np.concatenate(list_X), np.concatenate(list_Y)
        gmm = GaussianMixture(n_components = self.n_components, covariance_type = self.covariance_type, random_state=434)
        gmm.fit(marginal_Y)
        
        self.list_coef_ = [np.zeros(shape = (p_exog, K)) for i in range(K)]
        self.list_intercept_ = [np.zeros(shape = K) for i in range(K)]
        list_cur_list_trans_mat = self._est_trans_mat(list_exog)

        self.list_loglik_ = []
        for epoch in range(self.n_iter):

            # M-step
            if epoch != 0:
                tmp_weight_mat = []
                for cur_log_forward_prob, cur_log_backward_prob in zip(list_cur_log_forward_prob, list_cur_log_backward_prob):
                    tmp_log_weight_mat = cur_log_forward_prob + cur_log_backward_prob
                    tmp_log_weight_mat -= logsumexp(tmp_log_weight_mat[-1, :])
                    tmp_weight_mat.append(np.exp(tmp_log_weight_mat))
                tmp_weight_mat = np.concatenate(tmp_weight_mat)
            else:
                tmp_weight_mat = gmm.predict_proba(marginal_Y)
                
            list_lr_cov = []
            for ii in range(K):
                tmp_weight = tmp_weight_mat[:, ii]
                tmp_lr = self.reg_method(**self.kargs_reg)
                tmp_lr.fit(marginal_X, marginal_Y, sample_weight = tmp_weight)
                tmp_resid = marginal_Y - tmp_lr.predict(marginal_X)
                if self.covariance_type == "full":
                    tmp_cov_sum = np.zeros(shape = (p, p))
                    for jj in range(len(marginal_X)):
                        tmp_x = tmp_resid[jj, :]
                        tmp_cov_sum += np.outer(tmp_x, tmp_x) * tmp_weight[jj]
                    tmp_cov = tmp_cov_sum / np.sum(tmp_weight) 
                elif self.covariance_type == "diag":
                    tmp_cov = np.array([max(np.sum(tmp_weight * (tmp_resid[:, j] ** 2)) / np.sum(tmp_weight), 1e-12) for j in range(p)])
                list_lr_cov.append((tmp_lr, tmp_cov))
            self.list_lr_cov = list_lr_cov
            
            list_cur_mat_log_b = self._calc_emission_mat(list_X, list_Y)
                        
            if epoch != 0:
                cur_log_initprob = [list_cur_log_forward_prob[i][0] + list_cur_log_backward_prob[i][0] for i in range(n_seq)]
            else:
                cur_log_initprob = [[np.log(i) if i != 0 else -100 for i in line] for line in gmm.predict_proba([Y[0] for Y in list_Y])]
            
            cur_log_initprob = np.sum(cur_log_initprob, axis = 0)
            cur_log_initprob -= logsumexp(cur_log_initprob)
            self.startprob_ = np.exp(cur_log_initprob)
            self.log_startprob_ = cur_log_initprob

            # E-step      
            list_cur_log_forward_prob = [[] for i in range(n_seq)]
            for index_X, X in enumerate(list_X):
                for tt in range(len(X)):
                    if tt == 0:
                        tmp_log_prob = cur_log_initprob + list_cur_mat_log_b[index_X][0, :]
                    else:
                        tmp_log_prob = matmul_log(np.transpose(list_cur_list_trans_mat[index_X][tt - 1]), list_cur_log_forward_prob[index_X][-1]) + list_cur_mat_log_b[index_X][tt, :]
                    list_cur_log_forward_prob[index_X].append(tmp_log_prob)
            list_cur_log_forward_prob = [np.array(i) for i in list_cur_log_forward_prob]
            
            list_cur_log_backward_prob = [[0 for i in range(T)] for T in list_T]
            for index_X, (X, T) in enumerate(zip(list_X, list_T)):
                for tt in range(T - 1, -1, -1):
                    if tt == T - 1:
                        tmp_log_prob = np.array([np.log(1.) for i in range(K)])
                    else:
                        tmp_log_prob = []
                        for ii in range(K):
                            tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
                        tmp_log_prob = np.array(tmp_log_prob)
                    list_cur_log_backward_prob[index_X][tt] = tmp_log_prob
            list_cur_log_backward_prob = [np.array(i) for i in list_cur_log_backward_prob]
            
            list_tmp_array2d_log_gamma = [i + j for (i, j) in zip(list_cur_log_forward_prob, list_cur_log_backward_prob)]
            list_tmp_array1d_log_gamma_sum = [[logsumexp(i[t, :]) for t in range(len(i))] for i in list_tmp_array2d_log_gamma]
            
            list_tmp_array3d_log_epsilon = [np.zeros(shape = (T - 1, K, K)) for T in list_T]
            for index_X, (X, T) in enumerate(zip(list_X, list_T)):
                for tt in range(T - 1):
                    for ii in range(K):
                        for jj in range(K):
                            list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
                                    list_cur_log_backward_prob[index_X][tt + 1, jj] + list_cur_mat_log_b[index_X][tt + 1, jj] - list_tmp_array1d_log_gamma_sum[index_X][tt]            
        
            self._update_coef(list_tmp_array3d_log_epsilon, list_exog) # update coef and intercept for transmat
            list_cur_list_trans_mat = self._est_trans_mat(list_exog)
            self.list_cur_list_trans_mat = list_cur_list_trans_mat
            
            loglik = np.sum([logsumexp(i[-1]) for i in list_cur_log_forward_prob])
            self.list_loglik_.append(loglik)
            
            self.is_initialized = True
            
            if self.is_print_loglik:
                print("epoch % s, loglik = % s" % (epoch, loglik))
            
            if (len(self.list_loglik_) >= 2) and (abs(self.list_loglik_[-1] - self.list_loglik_[-2]) < self.tol_loglik):
                break
        
        self.list_last_posterior = [i[-1] + j[-1] for (i, j) in zip(list_cur_log_backward_prob, list_cur_log_forward_prob)]
        self.list_log_pred_posterior = []
            
        list_transmat_additional = self._est_trans_mat(list_exog_additional) if list_exog_additional is not None else None
        
        for ii in range(n_seq):
            tmp = self.list_last_posterior[ii]
            tmp -= logsumexp(tmp)
            self.list_last_posterior[ii] = np.exp(tmp)
            tmp_transmat = list_cur_list_trans_mat[ii][-1]
            if list_transmat_additional is not None:
                for tmp_transmat_additional in list_transmat_additional[ii]:
                    tmp_transmat = tmp_transmat @ tmp_transmat_additional
            self.list_log_pred_posterior.append(matmul_log(np.transpose(tmp_transmat), tmp))
            
        if list_X_test is None:
            pred = None
        else:
            pred = []
            for ii in range(n_seq):
                tmp_pred_prob = np.exp(self.list_log_pred_posterior[ii])
                tmp_pred = 0
                for tmp_k in range(K):
                    tmp_pred += list_lr_cov[tmp_k][0].predict([list_X_test[ii]])[0] * tmp_pred_prob[tmp_k]
                pred.append(tmp_pred)

            if not is_multiple_sequence:
                pred = pred[0]
        
        return pred
    
    
    def fit(self, X, Y, exog, is_multiple_sequence = False):
        self.fit_predict(X = X, Y = Y, exog = exog, X_test = None, is_multiple_sequence = is_multiple_sequence)
        return
    
    
    def _calc_emission_mat(self, list_X, list_Y):
        
        list_T = [len(i) for i in list_X]
        p, K = self.n_features, self.n_components
        
        list_cur_mat_log_b = [np.zeros(shape = (i, K)) for i in list_T]
        for ii, (tmp_lr, tmp_cov) in enumerate(self.list_lr_cov):
            for index_X, (X, Y) in enumerate(zip(list_X, list_Y)):
                tmp_resid = Y - tmp_lr.predict(X)
                if self.covariance_type == "full":
                    list_cur_mat_log_b[index_X][:, ii] = scipy.stats.multivariate_normal.logpdf(tmp_resid, mean=[0] * p, cov=tmp_cov)
                elif self.covariance_type == "diag":
                    tmp_logprob = 0
                    for jj in range(p):
                        tmp_logprob += scipy.stats.norm.logpdf(tmp_resid[:, jj], loc = 0, scale = np.sqrt(tmp_cov[jj]))
                    list_cur_mat_log_b[index_X][:, ii] = tmp_logprob

        return list_cur_mat_log_b
    
    
    def online_predict(self, X_test, Y_test, exog_test, is_multiple_sequence = False):
        
        if not self.is_fitted:
            print("Error: Not fitted. MSLRX's online_predict method must be used AFTER fit")
            return
            
        #matmul_log = lambda A, log_b: np.array([logsumexp(np.log(tmp_line) + log_b) for tmp_line in A])
        
        if is_multiple_sequence:
            list_X_test, list_Y_test = [np.array(i) for i in X_test], [np.array(i) for i in Y_test]  
            list_exog_test = [np.array(i) for i in exog_test] 
        else:
            list_X_test, list_Y_test = [np.array(X_test)], [np.array(Y_test)] 
            list_exog_test = [np.array(exog_test)] 

        list_T = [len(i) for i in list_Y_test]
        K = self.n_components
            
        if self.is_logistic_regression_standardized:
            list_exog_test = [self.standardizer.transform(i, copy=True) for i in list_exog_test]
        
        list_cur_mat_log_b = self._calc_emission_mat(list_X_test, list_Y_test)
        list_cur_list_trans_mat = self._est_trans_mat(list_exog_test)

        list_pred = []
        for ii, (cur_mat_log_b, cur_list_trans_mat) in enumerate(zip(list_cur_mat_log_b, list_cur_list_trans_mat)):
            pred = []
            log_last_posterior = self.list_log_pred_posterior[ii]
            for tt in range(list_T[ii]):
                
                log_last_posterior = log_last_posterior + cur_mat_log_b[tt, :]         
                log_last_posterior = matmul_log(np.transpose(cur_list_trans_mat[tt]), log_last_posterior)                    
                log_last_posterior -= logsumexp(log_last_posterior)
                tmp_posterior = np.exp(log_last_posterior)
                
                if tt + 1 < len(list_X_test[ii]):
                    tmp_mean_pred = 0
                    for tmp_k in range(K):
                        tmp_mean_pred += self.list_lr_cov[tmp_k][0].predict([list_X_test[ii][tt + 1]])[0] * tmp_posterior[tmp_k]
                    pred.append(tmp_mean_pred)
                
            self.list_log_pred_posterior[ii] = log_last_posterior
            pred = np.array(pred)
            list_pred.append(pred)
            
        if not is_multiple_sequence:
            list_pred = list_pred[0]
        list_pred = np.array(list_pred)

        res = list_pred
            
        return res
    
    
# def _test_MSLRX():
    
#     start_time = datetime.datetime.now()
#     print("start running _test_MSLRX()")
    
#     np.random.seed(434)
    
#     list_beta = [np.array([[0.5, 0.5], [0.5, 0.5], [0.5, 0.5]]), 
#                  np.array([[0.1, 0.5], [0.8, 0.5], [0.2, 0.6]]), 
#                  np.array([[-0.5, 0.7], [-0.5, -0.8], [-0.5, -0.2]]),]
#     list_intercept = [np.array([1., 2.]), np.array([8., 2.]), np.array([5., 3.])]
#     list_cov = [np.array([[1., 0.1], [0.1, 1.]]), np.array([[1.5, 0.1], [0.1, 1.5]]), np.array([[3., 0.1], [0.1, 3.]])]
#     transmat = np.array([[0.9, 0.05,  0.05], [ 0.3, 0.6,  0.1], [ 0.3,  0.2, 0.5]])
#     init_prob = np.array([1 / 3, 1 / 2, 1 / 6])
#     n_components = len(list_beta)
#     K = n_components
#     p_X = len(list_beta[0])
#     p_exog = 5
    
#     list_coef_transmat = [np.random.uniform(size = (p_exog, K), low = -1., high = 1.) for i in range(K)]
#     list_intercept_transmat = [np.random.uniform(size = K) for i in range(K)]
#     for line in list_coef_transmat:
#         line[0, :] = 0
    
#     T_train = 1000
#     T_test = 50
#     T = T_train + T_test
    
#     list_X, list_Y, list_exog = [], [], []
#     for ii in range(10):
        
#         X = np.random.normal(loc=1.0, scale=5.0, size=(T, p_X))
#         exog = np.random.normal(loc=0.0, scale=1.0, size = (T, p_exog))
#         Y, list_h_label = [], []
#         for tt in range(T):
            
#             index = tt
            
#             if index != 0:
#                 trans_mat = []
#                 for ii in range(K):
#                     tmp_log_prob = exog[index - 1] @ list_coef_transmat[ii] + list_intercept_transmat[ii]
#                     tmp_log_prob -= np.max(tmp_log_prob)
#                     tmp_prob = np.exp(tmp_log_prob)
#                     tmp_prob /= np.sum(tmp_prob)
#                     trans_mat.append(tmp_prob)
#                 trans_mat = np.array(trans_mat)
#                 prob_h_tmp = (h_label_prev @ trans_mat)[0]
#             else:
#                 prob_h_tmp = init_prob
            
#             h_label_tmp_vec = np.random.multinomial(n = 1, pvals = prob_h_tmp, size = 1)
#             h_label_tmp = np.argmax(h_label_tmp_vec)
#             tmp_h_label = h_label_tmp
#             h_label_prev = h_label_tmp_vec
            
#             list_h_label.append(tmp_h_label)
            
#             tmp_Y = X[tt, :] @ list_beta[tmp_h_label] + list_intercept[tmp_h_label] + np.random.multivariate_normal(mean = [0, 0], cov = list_cov[tmp_h_label])
#             Y.append(tmp_Y)
            
#         Y = np.array(Y)
        
#         list_X.append(X)
#         list_Y.append(Y)
#         list_exog.append(exog)
        
#     print("data prepared", str(datetime.datetime.now() - start_time))
        
#     mslrx = MSLRX(n_components=3, covariance_type="full", n_iter=10, is_logistic_regression_standardized = True)
#     pred = mslrx.fit_predict(X = [X[:T_train] for X in list_X],
#                              Y = [Y[:T_train] for Y in list_Y],
#                              exog = [exog[:T_train] for exog in list_exog],
#                              X_test = [X[T_train, :] for X in list_X], 
#                              is_multiple_sequence=True,
#                             exog_additional = [exog[:1] for X in list_X],
#                             )
#     print(pred)
#     ol_pred = mslrx.online_predict(X_test = [X[T_train:] for X in list_X],
#                                    Y_test = [Y[T_train:] for Y in list_Y], 
#                                    exog_test = [exog[T_train:] for exog in list_exog],
#                                    is_multiple_sequence = True)
    
#     list_pred = [[] for i in range(len(list_X))]
#     for cc, (tmp_pred, tmp_pred_ol) in enumerate(zip(pred, ol_pred)):
#         list_pred[cc].append(tmp_pred)
#         list_pred[cc].extend(tmp_pred_ol)

#     print("r2 =", r2_score(y_true = np.array([Y[T_train] for Y in list_Y]).flatten(), y_pred=np.array(pred).flatten()))
#     print("r2 =", r2_score(y_true = np.array([Y[T_train:] for Y in list_Y]).flatten(), y_pred=np.array(list_pred).flatten()))
#     print("running time =", str(datetime.datetime.now() - start_time))
    
#     return


# _test_MSLRX()


In [10]:
class MSLRXSoluIII():
    
    def __init__(self, n_components = 2, covariance_type="full", reg_method = "OLS", kargs_reg = None, n_iter = 10, tol_loglik = 0.):
        
        self.n_components = n_components
        self.covariance_type = covariance_type 
        self.n_iter = n_iter
        self.min_iter = 3
        self.tol_loglik = tol_loglik
        self.is_print_loglik = False
        
        if kargs_reg is None:
            kargs_reg = dict()
        self.kargs_reg = kargs_reg
            
        if reg_method in {"OLS", "LR", "LinearRegression", }:
            self.reg_method = LinearRegression
        elif reg_method in {"Ridge", }:
            self.reg_method = Ridge
        elif reg_method in {"RidgeCV", }:
            self.reg_method = RidgeCV
        elif reg_method in {"LinearSVR", "LinearSVM", }:
            self.reg_method = LinearSVR
        else:
            self.reg_method = reg_method
            
        return
    
    
    def _est_trans_mat(self, list_exog):
        
        list_list_trans_mat = []
        for exog in list_exog:
            T, K = len(exog), self.n_components
            list_trans_mat = []
            for tmp_exog in exog:
                tmp_trans_mat = []
                for ii in range(K):
                    tmp_prob_1 = tmp_exog @ self.list_coef_[ii] + self.list_intercept_[ii]
                    tmp_prob_1 = max(min(tmp_prob_1, 1 - 1e-5), 1e-5)
                    tmp_prob_0 = 1 - tmp_prob_1
                    tmp_trans_mat.append([tmp_prob_0, tmp_prob_1])
                list_trans_mat.append(np.array(tmp_trans_mat))
            list_list_trans_mat.append(np.array(list_trans_mat))
            
        return list_list_trans_mat
    
    
    def _update_coef(self, list_array3d_log_epsilon, list_exog):
        
        K = self.n_components
        self.list_coef_, self.list_intercept_ = [], []
        for ii in range(K):
            
            tmp_X, tmp_y, tmp_weight = [], [], []
            tmp_X_nan, tmp_y_nan, tmp_weight_nan = [], [], []
            for array3d_log_epsilon, exog in zip(list_array3d_log_epsilon, list_exog):
                T = len(array3d_log_epsilon)
                for tt in range(T):
                    for jj in range(K):
                        tmp_weight_cell = np.exp(array3d_log_epsilon[tt, ii, jj])
                        if np.isnan(tmp_weight_cell) or np.isinf(tmp_weight_cell):
                            tmp_X_nan.append(exog[tt])
                            tmp_y_nan.append(jj)
                            tmp_weight_nan.append(tmp_weight_cell)
                        else:
                            tmp_X.append(exog[tt])
                            tmp_y.append(jj)
                            tmp_weight.append(tmp_weight_cell)

            if len(tmp_y_nan) >= 1: 
                print("Warning: encountering % s nan in weight in ghmm_exog._update_coef" % len(tmp_y_nan))
                if len(set(tmp_y)) < K:
                    tmp_X, tmp_y, tmp_weight = np.array(tmp_X_nan), np.array(tmp_y_nan), np.array(tmp_weight_nan)
                    tmp_weight = 1
            
            tmp_X = sm.add_constant(tmp_X, has_constant='add', prepend=True)
            res_glm = sm.GLM(exog = tmp_X, endog = tmp_y, family=sm.families.Binomial(link = sm.families.links.identity()), freq_weights = tmp_weight).fit()
            
            tmp_intercept = res_glm.params[0]
            tmp_coef = np.array(res_glm.params[1:])
            
            self.list_coef_.append(tmp_coef) 
            self.list_intercept_.append(tmp_intercept)

        return
    
    
    def fit_predict(self, X, Y, exog, X_test, is_multiple_sequence = False, exog_additional = None):
        
        #matmul_log = lambda A, log_b: np.array([logsumexp(np.log(tmp_line) + log_b) for tmp_line in A])
        
        self.is_fitted = True
        self.is_initialized = False
        
        if is_multiple_sequence:
            list_X, list_Y = [np.array(i) for i in X], [np.array(i) for i in Y]
            list_X_test = [np.array(i) for i in X_test] if X_test is not None else None
            list_exog = [np.array(i) for i in exog]
            list_exog_additional = [np.array(i) for i in exog_additional] if exog_additional is not None else None
        else:
            list_X, list_Y = [np.array(X)], [np.array(Y)]
            list_X_test = [np.array(X_test)] if X_test is not None else None
            list_exog = [np.array(exog)]
            list_exog_additional = [np.array(exog_additional)]
        
        n_seq = len(list_Y)
        list_T = [len(i) for i in list_Y]
        
        p_X, p = len(list_X[0][0]), len(list_Y[0][0])
        p_exog = len(list_exog[0][0])
        K = self.n_components
        self.n_features = p
        
        marginal_X, marginal_Y = np.concatenate(list_X), np.concatenate(list_Y)
        gmm = GaussianMixture(n_components = self.n_components, covariance_type = self.covariance_type, random_state=434)
        gmm.fit(marginal_Y)
        
        self.list_coef_ = [np.zeros(shape = p_exog) for i in range(K)]
        self.list_intercept_ = [1 / K for i in range(K)]
        list_cur_list_trans_mat = self._est_trans_mat(list_exog)

        self.list_loglik_ = []
        for epoch in range(self.n_iter):

            # M-step
            if epoch != 0:
                tmp_weight_mat = []
                for cur_log_forward_prob, cur_log_backward_prob in zip(list_cur_log_forward_prob, list_cur_log_backward_prob):
                    tmp_log_weight_mat = cur_log_forward_prob + cur_log_backward_prob
                    tmp_log_weight_mat -= logsumexp(tmp_log_weight_mat[-1, :])
                    tmp_weight_mat.append(np.exp(tmp_log_weight_mat))
                tmp_weight_mat = np.concatenate(tmp_weight_mat)
            else:
                tmp_weight_mat = gmm.predict_proba(marginal_Y)
                
            list_lr_cov = []
            for ii in range(K):
                tmp_weight = tmp_weight_mat[:, ii]
                tmp_lr = self.reg_method(**self.kargs_reg)
                tmp_lr.fit(marginal_X, marginal_Y, sample_weight = tmp_weight)
                tmp_resid = marginal_Y - tmp_lr.predict(marginal_X)
                if self.covariance_type == "full":
                    tmp_cov_sum = np.zeros(shape = (p, p))
                    for jj in range(len(marginal_X)):
                        tmp_x = tmp_resid[jj, :]
                        tmp_cov_sum += np.outer(tmp_x, tmp_x) * tmp_weight[jj]
                    tmp_cov = tmp_cov_sum / np.sum(tmp_weight) 
                elif self.covariance_type == "diag":
                    tmp_cov = np.array([max(np.sum(tmp_weight * (tmp_resid[:, j] ** 2)) / np.sum(tmp_weight), 1e-12) for j in range(p)])
                list_lr_cov.append((tmp_lr, tmp_cov))
            self.list_lr_cov = list_lr_cov
            
            list_cur_mat_log_b = self._calc_emission_mat(list_X, list_Y)
                        
            if epoch != 0:
                cur_log_initprob = [list_cur_log_forward_prob[i][0] + list_cur_log_backward_prob[i][0] for i in range(n_seq)]
            else:
                cur_log_initprob = [[np.log(i) if i != 0 else -100 for i in line] for line in gmm.predict_proba([Y[0] for Y in list_Y])]
            
            cur_log_initprob = np.sum(cur_log_initprob, axis = 0)
            cur_log_initprob -= logsumexp(cur_log_initprob)
            self.startprob_ = np.exp(cur_log_initprob)
            self.log_startprob_ = cur_log_initprob

            # E-step      
            list_cur_log_forward_prob = [[] for i in range(n_seq)]
            for index_X, X in enumerate(list_X):
                for tt in range(len(X)):
                    if tt == 0:
                        tmp_log_prob = cur_log_initprob + list_cur_mat_log_b[index_X][0, :]
                    else:
                        tmp_log_prob = matmul_log(np.transpose(list_cur_list_trans_mat[index_X][tt - 1]), list_cur_log_forward_prob[index_X][-1]) + list_cur_mat_log_b[index_X][tt, :]
                    list_cur_log_forward_prob[index_X].append(tmp_log_prob)
            list_cur_log_forward_prob = [np.array(i) for i in list_cur_log_forward_prob]
            
            list_cur_log_backward_prob = [[0 for i in range(T)] for T in list_T]
            for index_X, (X, T) in enumerate(zip(list_X, list_T)):
                for tt in range(T - 1, -1, -1):
                    if tt == T - 1:
                        tmp_log_prob = np.array([np.log(1.) for i in range(K)])
                    else:
                        tmp_log_prob = []
                        for ii in range(K):
                            tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
                        tmp_log_prob = np.array(tmp_log_prob)
                    list_cur_log_backward_prob[index_X][tt] = tmp_log_prob
            list_cur_log_backward_prob = [np.array(i) for i in list_cur_log_backward_prob]
            
            list_tmp_array2d_log_gamma = [i + j for (i, j) in zip(list_cur_log_forward_prob, list_cur_log_backward_prob)]
            list_tmp_array1d_log_gamma_sum = [[logsumexp(i[t, :]) for t in range(len(i))] for i in list_tmp_array2d_log_gamma]
            
            list_tmp_array3d_log_epsilon = [np.zeros(shape = (T - 1, K, K)) for T in list_T]
            for index_X, (X, T) in enumerate(zip(list_X, list_T)):
                for tt in range(T - 1):
                    for ii in range(K):
                        for jj in range(K):
                            list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
                                    list_cur_log_backward_prob[index_X][tt + 1, jj] + list_cur_mat_log_b[index_X][tt + 1, jj] - list_tmp_array1d_log_gamma_sum[index_X][tt]            
        
            self._update_coef(list_tmp_array3d_log_epsilon, list_exog) # update coef and intercept for transmat
            list_cur_list_trans_mat = self._est_trans_mat(list_exog)
            self.list_cur_list_trans_mat = list_cur_list_trans_mat
            
            loglik = np.sum([logsumexp(i[-1]) for i in list_cur_log_forward_prob])
            self.list_loglik_.append(loglik)
            
            self.is_initialized = True
            
            if self.is_print_loglik:
                print("epoch % s, loglik = % s" % (epoch, loglik))
            
            if (len(self.list_loglik_) >= 2) and (abs(self.list_loglik_[-1] - self.list_loglik_[-2]) < self.tol_loglik):
                break
        
        self.list_last_posterior = [i[-1] + j[-1] for (i, j) in zip(list_cur_log_backward_prob, list_cur_log_forward_prob)]
        self.list_log_pred_posterior = []

        list_transmat_additional = self._est_trans_mat(list_exog_additional) if list_exog_additional is not None else None
        
        for ii in range(n_seq):
            tmp = self.list_last_posterior[ii]
            tmp -= logsumexp(tmp)
            self.list_last_posterior[ii] = np.exp(tmp)
            
            tmp_transmat = list_cur_list_trans_mat[ii][-1]
            if list_transmat_additional is not None:
                for tmp_transmat_additional in list_transmat_additional[ii]:
                    tmp_transmat = tmp_transmat @ tmp_transmat_additional
            self.list_log_pred_posterior.append(matmul_log(np.transpose(tmp_transmat), tmp))
            
        if list_X_test is None:
            pred = None
        else:
            pred = []
            for ii in range(n_seq):
                tmp_pred_prob = np.exp(self.list_log_pred_posterior[ii])
                tmp_pred = 0
                for tmp_k in range(K):
                    tmp_pred += list_lr_cov[tmp_k][0].predict([list_X_test[ii]])[0] * tmp_pred_prob[tmp_k]
                pred.append(tmp_pred)

            if not is_multiple_sequence:
                pred = pred[0]
        
        return pred
    
    
    def fit(self, X, Y, exog, is_multiple_sequence = False):
        self.fit_predict(X = X, Y = Y, exog = exog, X_test = None, is_multiple_sequence = is_multiple_sequence)
        return
    
    
    def _calc_emission_mat(self, list_X, list_Y):
        
        list_T = [len(i) for i in list_X]
        p, K = self.n_features, self.n_components
        
        list_cur_mat_log_b = [np.zeros(shape = (i, K)) for i in list_T]
        for ii, (tmp_lr, tmp_cov) in enumerate(self.list_lr_cov):
            for index_X, (X, Y) in enumerate(zip(list_X, list_Y)):
                tmp_resid = Y - tmp_lr.predict(X)
                if self.covariance_type == "full":
                    list_cur_mat_log_b[index_X][:, ii] = scipy.stats.multivariate_normal.logpdf(tmp_resid, mean=[0] * p, cov=tmp_cov)
                elif self.covariance_type == "diag":
                    tmp_logprob = 0
                    for jj in range(p):
                        tmp_logprob += scipy.stats.norm.logpdf(tmp_resid[:, jj], loc = 0, scale = np.sqrt(tmp_cov[jj]))
                    list_cur_mat_log_b[index_X][:, ii] = tmp_logprob

        return list_cur_mat_log_b
    
    
    def online_predict(self, X_test, Y_test, exog_test, is_multiple_sequence = False):
        
        if not self.is_fitted:
            print("Error: Not fitted. MSLRX's online_predict method must be used AFTER fit")
            return
            
        #matmul_log = lambda A, log_b: np.array([logsumexp(np.log(tmp_line) + log_b) for tmp_line in A])
        
        if is_multiple_sequence:
            list_X_test, list_Y_test = [np.array(i) for i in X_test], [np.array(i) for i in Y_test]  
            list_exog_test = [np.array(i) for i in exog_test] 
        else:
            list_X_test, list_Y_test = [np.array(X_test)], [np.array(Y_test)] 
            list_exog_test = [np.array(exog_test)] 

        list_T = [len(i) for i in list_Y_test]
        K = self.n_components
        
        list_cur_mat_log_b = self._calc_emission_mat(list_X_test, list_Y_test)
        list_cur_list_trans_mat = self._est_trans_mat(list_exog_test)

        list_pred = []
        for ii, (cur_mat_log_b, cur_list_trans_mat) in enumerate(zip(list_cur_mat_log_b, list_cur_list_trans_mat)):
            pred = []
            log_last_posterior = self.list_log_pred_posterior[ii]
            for tt in range(list_T[ii]):
                
                log_last_posterior = log_last_posterior + cur_mat_log_b[tt, :]         
                log_last_posterior = matmul_log(np.transpose(cur_list_trans_mat[tt]), log_last_posterior)                    
                log_last_posterior -= logsumexp(log_last_posterior)
                tmp_posterior = np.exp(log_last_posterior)
                
                if tt + 1 < len(list_X_test[ii]):
                    tmp_mean_pred = 0
                    for tmp_k in range(K):
                        tmp_mean_pred += self.list_lr_cov[tmp_k][0].predict([list_X_test[ii][tt + 1]])[0] * tmp_posterior[tmp_k]
                    pred.append(tmp_mean_pred)
                
            self.list_log_pred_posterior[ii] = log_last_posterior
            pred = np.array(pred)
            list_pred.append(pred)
            
        if not is_multiple_sequence:
            list_pred = list_pred[0]
        list_pred = np.array(list_pred)

        res = list_pred
            
        return res
    
    
# def _test_MSLRXSoluIII():
    
#     start_time = datetime.datetime.now()
#     print("start running _test_MSLRXSoluIII()")
    
#     np.random.seed(434)
    
#     list_beta = [np.array([[0.5, 0.5], [0.5, 0.5], [0.5, 0.5]]), 
#                  np.array([[-0.5, 0.7], [-0.5, -0.8], [-0.5, -0.2]]),]
#     list_intercept = [np.array([1., 2.]), np.array([8., 2.])]
#     list_cov = [np.array([[1., 0.1], [0.1, 1.]]), np.array([[3., 0.1], [0.1, 3.]])]
#     transmat = np.array([[0.9, 0.1], [ 0.3, 0.7]])
#     init_prob = np.array([1 / 3, 2 / 3])
#     n_components = len(list_beta)
#     K = n_components
#     p_X = len(list_beta[0])
#     p_exog = 5
    
#     list_coef_transmat = [np.random.uniform(size = (p_exog, K), low = -1., high = 1.) for i in range(K)]
#     list_intercept_transmat = [np.random.uniform(size = K) for i in range(K)]
#     for line in list_coef_transmat:
#         line[0, :] = 0
    
#     T_train = 1000
#     T_test = 50
#     T = T_train + T_test
    
#     list_X, list_Y, list_exog = [], [], []
#     for ii in range(100):
        
#         X = np.random.normal(loc=1.0, scale=5.0, size=(T, p_X))
#         exog = np.random.normal(loc=0.0, scale=1.0, size = (T, p_exog))
#         Y, list_h_label = [], []
#         for tt in range(T):
            
#             index = tt
            
#             if index != 0:
#                 trans_mat = []
#                 for ii in range(K):
#                     tmp_log_prob = exog[index - 1] @ list_coef_transmat[ii] + list_intercept_transmat[ii]
#                     tmp_log_prob -= np.max(tmp_log_prob)
#                     tmp_prob = np.exp(tmp_log_prob)
#                     tmp_prob /= np.sum(tmp_prob)
#                     trans_mat.append(tmp_prob)
#                 trans_mat = np.array(trans_mat)
#                 prob_h_tmp = (h_label_prev @ trans_mat)[0]
#             else:
#                 prob_h_tmp = init_prob
            
#             h_label_tmp_vec = np.random.multinomial(n = 1, pvals = prob_h_tmp, size = 1)
#             h_label_tmp = np.argmax(h_label_tmp_vec)
#             tmp_h_label = h_label_tmp
#             h_label_prev = h_label_tmp_vec
            
#             list_h_label.append(tmp_h_label)
            
#             tmp_Y = X[tt, :] @ list_beta[tmp_h_label] + list_intercept[tmp_h_label] + np.random.multivariate_normal(mean = [0, 0], cov = list_cov[tmp_h_label])
#             Y.append(tmp_Y)
            
#         Y = np.array(Y)
        
#         list_X.append(X)
#         list_Y.append(Y)
#         list_exog.append(exog)
        
#     print("data prepared", str(datetime.datetime.now() - start_time))
        
#     mslrx = MSLRXSoluIII(n_components=2, covariance_type="full", n_iter=10)
#     pred = mslrx.fit_predict(X = [X[:T_train] for X in list_X],
#                              Y = [Y[:T_train] for Y in list_Y],
#                              exog = [exog[:T_train] for exog in list_exog],
#                              X_test = [X[T_train, :] for X in list_X], 
#                              is_multiple_sequence=True,
#                             exog_additional=[exog[:1] for X in list_X])
#     print(pred)
#     ol_pred = mslrx.online_predict(X_test = [X[T_train:] for X in list_X],
#                                    Y_test = [Y[T_train:] for Y in list_Y], 
#                                    exog_test = [exog[T_train:] for exog in list_exog],
#                                    is_multiple_sequence = True)
    
#     list_pred = [[] for i in range(len(list_X))]
#     for cc, (tmp_pred, tmp_pred_ol) in enumerate(zip(pred, ol_pred)):
#         list_pred[cc].append(tmp_pred)
#         list_pred[cc].extend(tmp_pred_ol)

#     print("r2 =", r2_score(y_true = np.array([Y[T_train] for Y in list_Y]).flatten(), y_pred=np.array(pred).flatten()))
#     print("r2 =", r2_score(y_true = np.array([Y[T_train:] for Y in list_Y]).flatten(), y_pred=np.array(list_pred).flatten()))
#     print("running time =", str(datetime.datetime.now() - start_time))
    
#     return


# _test_MSLRXSoluIII()


In [11]:
class SMap:
    
    def __init__(self, theta = 0., reg_method = "OLS", kargs_reg = None):
        
        self.theta = theta
        
        if kargs_reg is None:
            kargs_reg = dict()
        self.kargs_reg = kargs_reg
            
        if reg_method in {"OLS", "LR", "LinearRegression", }:
            self.reg_method = LinearRegression
        elif reg_method in {"Ridge", }:
            self.reg_method = Ridge
        elif reg_method in {"RidgeCV", }:
            self.reg_method = RidgeCV
        elif reg_method in {"LinearSVR", "LinearSVM", }:
            self.reg_method = LinearSVR
        else:
            self.reg_method = reg_method
            
        return
    
    
    def fit(self, X, y, sample_weight = None):
        self.X, self.y = np.array(X), np.array(y)
        if sample_weight is None:
            self.sample_weight = np.array([1. for i in range(len(self.X))])
        else:
            self.sample_weight = np.array(sample_weight)
        return
    
    
    def predict(self, X):
        
        X_train, y_train = self.X, self.y
        sample_weight_train = self.sample_weight
        theta = self.theta
        reg_method, kargs_reg = self.reg_method, self.kargs_reg
        
        y_pred = []
        for tmp_X_test in X:
            tmp_X_test = np.array(tmp_X_test)
            tmp_d_vec = np.sqrt(np.sum((X_train - tmp_X_test) ** 2, axis = 1))
            tmp_sample_weight = np.exp(- theta * tmp_d_vec / tmp_d_vec.mean())
            tmp_sample_weight *= sample_weight_train
            tmp_reg = reg_method(**kargs_reg)
            tmp_reg.fit(X_train, y_train, sample_weight = tmp_sample_weight)
            tmp_pred = tmp_reg.predict([tmp_X_test])[0]
            y_pred.append(tmp_pred)
            
        y_pred = np.array(y_pred)
        return y_pred
            
        
# def _test_SMap():
    
#     N, p, n_test = 10000, 5, 1000
#     beta = np.random.uniform(size = p)
#     X = np.random.normal(size = (N, p))
#     y = X @ beta + 2 + np.random.normal(size = N)
    
#     reg = SMap(theta = 0.5)
#     reg.fit(X[:-n_test], y[:-n_test])
#     y_pred = reg.predict(X[-n_test:])
#     print("r2 =", r2_score(y_true=y[-n_test:], y_pred = y_pred))
    
#     return
    
    
# _test_SMap()        
        

In [12]:
class SMapCV:
    
    def __init__(self, thetas = (0.0, 0.5, 1.0, 1.5, 2.0, ), reg_method = "OLS", kargs_reg = None):
        
        self.thetas = thetas
        
        if kargs_reg is None:
            kargs_reg = dict()
        self.kargs_reg = kargs_reg
            
        if reg_method in {"OLS", "LR", "LinearRegression", }:
            self.reg_method = LinearRegression
        elif reg_method in {"Ridge", }:
            self.reg_method = Ridge
        elif reg_method in {"RidgeCV", }:
            self.reg_method = RidgeCV
        elif reg_method in {"LinearSVR", "LinearSVM", }:
            self.reg_method = LinearSVR
        else:
            self.reg_method = reg_method
            
        return
    
    
    def fit(self, X, y, sample_weight = None):
        
        X, y = np.array(X), np.array(y)
        self.X, self.y = X, y
        if sample_weight is None:
            sample_weight = np.array([1. for i in range(len(self.X))])
        else:
            sample_weight = np.array(sample_weight)
        self.sample_weight = sample_weight
        
        dict_theta_pred = {i: [] for i in self.thetas}
        y_true = []
        sample_weight_true = []
        
        kf = KFold(n_splits=10, shuffle = True, random_state = 434)
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            sample_weight_train, sample_weight_test = sample_weight[train_index], sample_weight[test_index]
            
            y_true.extend(y_test)
            sample_weight_true.extend(sample_weight_test)
            
            for theta in self.thetas:
                tmp_smap = SMap(theta = theta, reg_method = self.reg_method, kargs_reg = self.kargs_reg)
                tmp_smap.fit(X_train, y_train, sample_weight = sample_weight_train)
                y_pred = tmp_smap.predict(X_test)
                dict_theta_pred[theta].extend(y_pred)
                
        list_theta_r2 = []
        for theta in self.thetas:
            tmp_r2 = r2_score(y_true = y_true, y_pred = dict_theta_pred[theta], sample_weight = sample_weight_true)
            list_theta_r2.append((theta, tmp_r2))
        list_theta_r2.sort(key = lambda x: x[1], reverse = True)        
        theta_opt = list_theta_r2[0][0]
        
        self.theta = theta_opt
        
        return
    
    
    def predict(self, X):
        
        X_train, y_train = self.X, self.y
        theta = self.theta
        reg_method, kargs_reg = self.reg_method, self.kargs_reg
        
        y_pred = []
        for tmp_X_test in X:
            tmp_X_test = np.array(tmp_X_test)
            tmp_d_vec = np.sqrt(np.sum((X_train - tmp_X_test) ** 2, axis = 1))
            tmp_sample_weight = np.exp(- theta * tmp_d_vec / tmp_d_vec.mean())
            tmp_reg = reg_method(**kargs_reg)
            tmp_reg.fit(X_train, y_train, sample_weight = tmp_sample_weight)
            tmp_pred = tmp_reg.predict([tmp_X_test])[0]
            y_pred.append(tmp_pred)
            
        y_pred = np.array(y_pred)
        return y_pred
            
        
# def _test_SMapCV():
    
#     N, p, n_test = 1000, 5, 100
#     beta = np.random.uniform(size = p)
#     X = np.random.normal(size = (N, p))
#     y = X @ beta + 2 + np.random.normal(size = N)
    
#     reg = SMapCV()
#     reg.fit(X[:-n_test], y[:-n_test])
#     y_pred = reg.predict(X[-n_test:])
#     print("theta = ", reg.theta)
#     print("r2 =", r2_score(y_true=y[-n_test:], y_pred = y_pred))
    
#     return
    
    
# _test_SMapCV()        
        

## Predictions (WW)

In [43]:
data_WW_byday = data_WW[["date", "conductivity", "turbidity"]].resample("1D").max()
data_WW_byday["temperature"] = data_WW["temperature"].resample("1D").mean()
data_WW_byday["pH"] = data_WW["pH"].resample("1D").mean()
data_WW_byday["ODO"] = data_WW["ODO"].resample("1D").min()
data_WW_byday["salinity_max"] = data_WW["salinity"].resample("1D").max()
data_WW_byday["salinity_min"] = data_WW["salinity"].resample("1D").min()
data_WW_byday["log10_chlorophyll"] = data_WW["log10_chlorophyll"].resample("1D").max()
data_WW_byday["chlorophyll"] = data_WW["chlorophyll"].resample("1D").max()

# data_WW_byday = data_WW[["temperature", "pH"]].resample("1D").mean()
# data_WW_byday["date"] = data_WW["date"].resample("1D").max()
# data_WW_byday["conductivity"] = data_WW["conductivity"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_WW_byday["turbidity"] = data_WW["turbidity"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_WW_byday["ODO"] = data_WW["ODO"].resample("1D").apply(lambda x: x.nsmallest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_WW_byday["salinity_max"] = data_WW["salinity"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_WW_byday["salinity_min"] = data_WW["salinity"].resample("1D").apply(lambda x: x.nsmallest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_WW_byday["log10_chlorophyll"] = data_WW["log10_chlorophyll"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_WW_byday["chlorophyll"] = data_WW["chlorophyll"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)


In [44]:
data_W_byday = data_W[["date"]].resample("1D").max()
#data_W_byday["airpressure"] = data_W["airpressure"].resample("1D").mean()
data_W_byday["wind"] = data_W["wind"].resample("1D").mean()

data_WW_byday = data_WW_byday.merge(data_W_byday, left_index=True, right_index=True).rename(columns = {"date_x": "date"}).drop(columns = ["date_y"])


In [45]:
data_WW_byday = data_WW_byday.dropna().resample("1D").max()

In [46]:
data_WW_byday

,date,conductivity,turbidity,temperature,pH,ODO,salinity_max,salinity_min,log10_chlorophyll,chlorophyll,wind
time_min,,,,,,,,,,,
2018-10-12,2018-10-12,46.978,47.06,24.194146,7.902439,5.84,30.55,28.48,1.295567,19.75,2.774167
2018-10-13,2018-10-13,46.748,29.83,22.100917,7.863437,5.89,30.41,28.41,1.480438,30.23,2.256528
2018-10-14,2018-10-14,46.615,18.17,20.642937,7.871354,6.06,30.33,28.59,1.342620,22.01,1.743472
2018-10-15,2018-10-15,46.385,15.75,21.330021,7.870104,6.32,30.16,28.72,1.330008,21.38,4.257870
2018-10-16,2018-10-16,46.521,18.81,21.417677,7.843646,5.63,30.26,28.59,1.221675,16.66,2.117500
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-20,2022-12-20,49.115,5.79,5.802531,7.942708,9.53,31.56,28.73,0.429752,2.69,1.510417
2022-12-21,2022-12-21,49.668,9.33,5.474958,7.948021,9.63,31.91,29.18,0.459392,2.88,2.171071
2022-12-22,2022-12-22,49.776,50.00,5.977583,7.944375,9.74,32.03,29.50,0.563481,3.66,4.545590


In [49]:
def _main(data_WW_byday, is_log = True, is_cross = True, is_phase = True, horizon_forecast = 1, thres_quantile = 0.95, n_seq_warmup = 6,
          reg_method = "LinearSVR", reg_method_func = LinearSVR, kargs_reg = {"random_state": 434},
          #reg_method = "LinearRegression", reg_method_func = LinearRegression, kargs_reg = {},
         ):
    
    # Data Preparation
    if is_log:
        array_obs = data_WW_byday["log10_chlorophyll"].to_numpy()
    else:
        array_obs = data_WW_byday["chlorophyll"].to_numpy()

    array_datetime = data_WW_byday["date"].to_numpy()
    array_cross = data_WW_byday[["salinity_max", "salinity_min", "ODO", "turbidity", "pH", "temperature"]].to_numpy()
    #array_cross = data_WW_byday[["salinity_max", "salinity_min", "ODO", "turbidity", "pH", "temperature", "wind"]].to_numpy()
    
    list_split_index = list(np.where(np.isnan(array_obs))[0])
    list_split_index.append(len(array_obs))
    list_split_index.insert(0, 0)
    list_subseq, list_subseq_cross,list_subseq_datetime = [], [], []
    for tmp_index in range(len(list_split_index) - 1):
        tmp_index_left = list_split_index[tmp_index] + 1
        tmp_index_right = list_split_index[tmp_index + 1]
        if tmp_index_right - tmp_index_left < 21:
            continue
        list_subseq.append(array_obs[tmp_index_left:tmp_index_right])
        list_subseq_datetime.append(array_datetime[tmp_index_left:tmp_index_right])
        list_subseq_cross.append(array_cross[tmp_index_left:tmp_index_right])
    
    list_p_AR = [2, 1, ]
    list_p_AR_cross = [1, ]
    list_p_AR.sort(reverse = True)
    max_p = max(max(list_p_AR), max(list_p_AR_cross) if len(list_p_AR_cross) != 0 else 0)

    list_X, list_Y, list_exog = [], [], []
    list_date = []
    for tmp_subseq, tmp_subseq_datetime, tmp_subseq_cross in zip(list_subseq, list_subseq_datetime, list_subseq_cross):
        X, Y, exog = [], [], []
        tmp_list_date = []
        for tmp_index_right in range(max_p, len(tmp_subseq) - horizon_forecast + 1):

            tmp_index_test = tmp_index_right - 1 + horizon_forecast # tmp_index_right == tmp_index_test if horizon_forecast = 1

            tmp_X = []
            if is_cross:
                for tmp_p in list_p_AR_cross:
                    tmp_X.extend(tmp_subseq_cross[tmp_index_right - tmp_p])
            for tmp_p in list_p_AR:
                tmp_X.append(tmp_subseq[tmp_index_right - tmp_p])
            X.append(tmp_X)
            Y.append([tmp_subseq[tmp_index_test]])

            tmp_date= tmp_subseq_datetime[tmp_index_test]
            tmp_list_date.append(tmp_date)
            
            tmp_t = pd.Timestamp(tmp_subseq_datetime[tmp_index_test]).toordinal() - pd.Timestamp(tmp_subseq_datetime[tmp_index_test]).replace(month = 1, day = 1).toordinal()
            if is_phase:
                tmp_exog = [np.cos(tmp_t / 365 * 2 * np.pi), np.sin(tmp_t / 365 * 2 * np.pi)]
            else:
                tmp_exog = [np.cos(tmp_t / 365 * 2 * np.pi)]
            exog.append(tmp_exog)

        list_X.append(X)
        list_Y.append(Y)
        list_exog.append(exog)
        list_date.append(tmp_list_date)
    
    print("Subseqs:")
    for tmp_index in range(len(list_subseq)):
        print("\t", tmp_index, pd.Timestamp(list_subseq_datetime[tmp_index][1]).date(), pd.Timestamp(list_subseq_datetime[tmp_index][-1]).date(), len(list_subseq[tmp_index]))
    print()
    
    # Prediction
    
    start_time = datetime.datetime.now()

    list_y_true, list_y_pred_naive = [], []
    list_y_pred_mslrx, list_y_pred_mslr = [], []
    list_y_pred_mslrxsolu3 = []
    list_y_date = []
    for cc in range(n_seq_warmup, len(list_X)):
        tmp_list_y_true, tmp_list_y_pred_naive = [], []
        tmp_list_y_pred_mslrx, tmp_list_y_pred_mslr = [], []
        tmp_list_y_pred_mslrxsolu3 = []
        tmp_list_y_date = []
        for tmp_index_test in range(5, len(list_X[cc])):
            tmp_list_X, tmp_list_Y, tmp_list_exog = [], [], []
            tmp_list_X_test = []
            for ii in range(cc):
                tmp_list_X.append(copy.deepcopy(list_X[ii]))
                tmp_list_Y.append(copy.deepcopy(list_Y[ii]))
                tmp_list_exog.append(copy.deepcopy(list_exog[ii]))
                tmp_list_X_test.append(copy.deepcopy(list_X[ii][-1]))
            tmp_list_X.append(copy.deepcopy(list_X[cc][:tmp_index_test]))
            tmp_list_Y.append(copy.deepcopy(list_Y[cc][:tmp_index_test]))
            tmp_list_exog.append(copy.deepcopy(list_exog[cc][:tmp_index_test]))
            tmp_list_X_test.append(copy.deepcopy(list_X[cc][tmp_index_test]))

            mslrx = MSLRX(n_components = 2, covariance_type="diag", n_iter = 10, is_logistic_regression_CV = False,
                          logistic_regression_C = 1e10, is_logistic_regression_standardized = False, reg_method = reg_method, kargs_reg = kargs_reg)
            tmp_pred = mslrx.fit_predict(tmp_list_X, tmp_list_Y, tmp_list_exog, tmp_list_X_test, is_multiple_sequence = True)
            tmp_y_pred = tmp_pred[-1]
            tmp_list_y_pred_mslrx.append(tmp_y_pred)

            mslrx = MSLRXSoluIII(n_components = 2, covariance_type="diag", n_iter = 10, reg_method = reg_method, kargs_reg = kargs_reg)
            tmp_pred = mslrx.fit_predict(tmp_list_X, tmp_list_Y, tmp_list_exog, tmp_list_X_test, is_multiple_sequence = True)
            tmp_y_pred = tmp_pred[-1]
            tmp_list_y_pred_mslrxsolu3.append(tmp_y_pred)

            mslr = MSLR(n_components = 2, covariance_type="diag", n_iter = 10, reg_method = reg_method, kargs_reg = kargs_reg)
            tmp_pred = mslr.fit_predict(tmp_list_X, tmp_list_Y, tmp_list_X_test, is_multiple_sequence = True)
            tmp_y_pred = tmp_pred[-1]
            tmp_list_y_pred_mslr.append(tmp_y_pred)

            tmp_y_true = list_Y[cc][tmp_index_test]
            #tmp_y_pred_naive = list_Y[cc][tmp_index_test - 1]
            tmp_y_pred_naive = list_X[cc][tmp_index_test][-1]
            tmp_list_y_true.append(tmp_y_true)
            tmp_list_y_pred_naive.append(tmp_y_pred_naive)
            tmp_list_y_date.append(list_date[cc][tmp_index_test])

        list_y_true.append(tmp_list_y_true)
        list_y_pred_naive.append(tmp_list_y_pred_naive)
        list_y_pred_mslrx.append(tmp_list_y_pred_mslrx)
        list_y_pred_mslr.append(tmp_list_y_pred_mslr)
        list_y_pred_mslrxsolu3.append(tmp_list_y_pred_mslrxsolu3)
        list_y_date.append(tmp_list_y_date)

        print("seq % s finished" % cc, str(datetime.datetime.now() - start_time))

        #if cc > 5: break
        
    if is_log:
        thres = np.nanquantile(np.power(10, array_obs), thres_quantile)
    else:
        thres = np.nanquantile(array_obs, thres_quantile)

    list_p_AR = [2, 1, ]
    list_p_AR_cross = [1, ]
    list_p_AR.sort(reverse = True)
    
    if is_cross:
        max_p = max(max(list_p_AR), max(list_p_AR_cross) if len(list_p_AR_cross) != 0 else 0)
    else:
        max_p = max(list_p_AR)
    
    dict_warmup_evaluation = dict()
    for n_seq_exclude_valid in range(n_seq_warmup, len(list_subseq)):
        
        print()
        print("n_seq_warmup =", n_seq_exclude_valid)
        
        list_res_logitcos_cross = []
        list_res_pred = []
        for start_index in range(7, 20):

            print(start_index)

            X, Y = [], []
            list_index_test_set = []
            list_date_test = []
            for cc, (tmp_subseq, tmp_subseq_datetime, tmp_subseq_cross) in enumerate(zip(list_subseq, list_subseq_datetime, list_subseq_cross)):
                for tmp_index_right in range(max_p, len(tmp_subseq) - horizon_forecast + 1):
                    tmp_index_test = tmp_index_right - 1 + horizon_forecast # tmp_index_right == tmp_index_test if horizon_forecast = 1
                    tmp_X = []
                    if is_cross:
                        for tmp_p in list_p_AR_cross:
                            tmp_X.extend(tmp_subseq_cross[tmp_index_right - tmp_p])
                    for tmp_p in list_p_AR:
                        tmp_X.append(tmp_subseq[tmp_index_right - tmp_p])
                    X.append(tmp_X)
                    Y.append([tmp_subseq[tmp_index_test]])

                    if cc >= n_seq_exclude_valid and tmp_index_right >= start_index:
                        list_index_test_set.append(len(Y) - 1)
                        list_date_test.append(tmp_subseq_datetime[tmp_index_test])

            y_true, y_pred = [], []
            y_pred_naive = []
            for tmp_index in list_index_test_set:
                lr = reg_method_func(**kargs_reg)
                #lr = SMap(theta = 0.5, reg_method = reg_method, kargs_reg = kargs_reg)
                #lr = SMap(theta = 0.5)
                lr.fit(X[:tmp_index], Y[:tmp_index])
                tmp_y_pred = lr.predict([X[tmp_index]])[0]
                tmp_y_true = Y[tmp_index]
                tmp_y_pred_naive = X[tmp_index][-1]
                y_true.append(tmp_y_true)
                y_pred.append(tmp_y_pred)
                y_pred_naive.append(tmp_y_pred_naive)

            if is_log:
                y_true = np.power(10, y_true)
                y_pred_naive = np.power(10, y_pred_naive)
                y_pred = np.power(10, y_pred)

            tmp_r2_ar, tmp_r2_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive)
            tmp_rocauc_ar, tmp_rocauc_naive = roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
            y_pred_ar = y_pred
            print("AR", tmp_r2_ar, tmp_r2_naive, tmp_rocauc_ar, tmp_rocauc_naive)
            
            y_true, y_pred = [], []
            y_pred_naive = []
            for tmp_index in list_index_test_set:
                #lr = reg_method_func(**kargs_reg)
                lr = SMap(theta = 0.5, reg_method = reg_method, kargs_reg = kargs_reg)
                #lr = SMap(theta = 0.5)
                lr.fit(X[:tmp_index], Y[:tmp_index])
                tmp_y_pred = lr.predict([X[tmp_index]])[0]
                tmp_y_true = Y[tmp_index]
                tmp_y_pred_naive = X[tmp_index][-1]
                y_true.append(tmp_y_true)
                y_pred.append(tmp_y_pred)
                y_pred_naive.append(tmp_y_pred_naive)

            if is_log:
                y_true = np.power(10, y_true)
                y_pred_naive = np.power(10, y_pred_naive)
                y_pred = np.power(10, y_pred)

            tmp_r2_smap, tmp_r2_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive)
            tmp_rocauc_smap, tmp_rocauc_naive = roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
            y_pred_smap = y_pred
            print("SMap", tmp_r2_smap, tmp_r2_naive, tmp_rocauc_smap, tmp_rocauc_naive)

            y_true, y_pred = [], []
            y_pred_naive = []
            for cc in range(n_seq_exclude_valid - n_seq_warmup, len(list_y_true)):
                for tmp_index_test in range(start_index - 5 - max_p, len(list_y_true[cc])):
                    y_true.append(list_y_true[cc][tmp_index_test])
                    y_pred.append(list_y_pred_mslrx[cc][tmp_index_test])
                    y_pred_naive.append(list_y_pred_naive[cc][tmp_index_test])

            if is_log:
                y_true = np.power(10, y_true)
                y_pred_naive = np.power(10, y_pred_naive)
                y_pred = np.power(10, y_pred)

            tmp_r2_mslrx, tmp_r2_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive)
            tmp_rocauc_mslrx, tmp_rocauc_naive = roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
            y_pred_mslrx = y_pred
            print("MSLRX", tmp_r2_mslrx, tmp_r2_naive, tmp_rocauc_mslrx, tmp_rocauc_naive)

            y_true, y_pred = [], []
            y_pred_naive = []
            for cc in range(n_seq_exclude_valid - n_seq_warmup, len(list_y_true)):
                for tmp_index_test in range(start_index - 5 - max_p, len(list_y_true[cc])):
                    y_true.append(list_y_true[cc][tmp_index_test])
                    y_pred.append(list_y_pred_mslrxsolu3[cc][tmp_index_test])
                    y_pred_naive.append(list_y_pred_naive[cc][tmp_index_test])

            if is_log:
                y_true = np.power(10, y_true)
                y_pred_naive = np.power(10, y_pred_naive)
                y_pred = np.power(10, y_pred)

            tmp_r2_mslrxsolu3, tmp_r2_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive)
            tmp_rocauc_mslrxsolu3, tmp_rocauc_naive = roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
            y_pred_mslrxsolu3 = y_pred
            print("MXLRXSolu3", tmp_r2_mslrxsolu3, tmp_r2_naive, tmp_rocauc_mslrxsolu3, tmp_rocauc_naive)

            y_true, y_pred = [], []
            y_pred_naive = []
            for cc in range(n_seq_exclude_valid - n_seq_warmup, len(list_y_true)):
                for tmp_index_test in range(start_index - 5 - max_p, len(list_y_true[cc])):
                    y_true.append(list_y_true[cc][tmp_index_test])
                    y_pred.append(list_y_pred_mslr[cc][tmp_index_test])
                    y_pred_naive.append(list_y_pred_naive[cc][tmp_index_test])

            if is_log:
                y_true = np.power(10, y_true)
                y_pred_naive = np.power(10, y_pred_naive)
                y_pred = np.power(10, y_pred)

            tmp_r2_mslr, tmp_r2_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive)
            tmp_rocauc_mslr, tmp_rocauc_naive = roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
            y_pred_mslr = y_pred
            print("MSLR", tmp_r2_mslr, tmp_r2_naive, tmp_rocauc_mslr, tmp_rocauc_naive)

            list_res_logitcos_cross.append((tmp_r2_naive, tmp_rocauc_naive,
                                            tmp_r2_ar, tmp_rocauc_ar, 
                                            tmp_r2_smap, tmp_rocauc_smap,
                                            tmp_r2_mslrx, tmp_rocauc_mslrx, 
                                            tmp_r2_mslrxsolu3, tmp_rocauc_mslrxsolu3, 
                                            tmp_r2_mslr, tmp_rocauc_mslr,
                                           ))
            
            list_res_pred.append((list_date_test, y_true, y_pred_naive, y_pred_ar, y_pred_smap, y_pred_mslrx, y_pred_mslrxsolu3, y_pred_mslr))
            
        dict_warmup_evaluation[n_seq_exclude_valid] = (list_res_logitcos_cross, list_res_pred)
                
    return dict_warmup_evaluation


In [50]:

dict_setting_results = dict()
for is_log in [True, False]:
    for is_cross in [True, False]:
        for is_phase in [True]:
            for horizon_forecast in [1, 3, 7]:
                
                tmp_setting = (is_log, is_cross, is_phase, horizon_forecast, "LinearRegression", )
                
                print()
                print("================================================")
                print("setting =", tmp_setting)
                print("================================================")
                
                dict_setting_results[tmp_setting] = _main(data_WW_byday, n_seq_warmup=1, 
                                                          #reg_method = "LinearSVR", reg_method_func = LinearSVR, kargs_reg = {"random_state": 434},
                                                          reg_method = "LinearRegression", reg_method_func = LinearRegression, kargs_reg = {},
                                                          is_log = is_log, is_cross = is_cross, is_phase = is_phase, horizon_forecast = horizon_forecast,)
                 
                



setting = (True, True, True, 1, 'LinearRegression')
Subseqs:
	 0 2018-11-01 2019-01-22 84
	 1 2019-02-02 2019-03-24 52
	 2 2019-03-27 2019-06-24 91
	 3 2019-06-27 2019-07-25 30
	 4 2019-08-02 2019-11-16 108
	 5 2019-11-20 2020-02-12 86
	 6 2020-02-15 2020-03-27 43
	 7 2020-03-31 2020-07-16 109
	 8 2020-09-04 2020-09-25 23
	 9 2021-02-09 2021-03-07 28
	 10 2021-04-01 2021-06-28 90
	 11 2021-07-01 2021-07-21 22
	 12 2021-07-29 2021-08-31 35
	 13 2021-09-03 2021-10-27 56
	 14 2021-10-30 2022-01-01 65
	 15 2022-01-15 2022-03-26 72
	 16 2022-03-29 2022-07-23 118
	 17 2022-07-28 2022-12-09 136

seq 1 finished 0:01:53.279489
seq 2 finished 0:06:13.645465
seq 3 finished 0:07:38.937970
seq 4 finished 0:15:23.656240
seq 5 finished 0:22:54.580526
seq 6 finished 0:26:56.567881
seq 7 finished 0:39:02.337595
seq 8 finished 0:41:04.477622
seq 9 finished 0:43:51.123879
seq 10 finished 0:55:12.846881
seq 11 finished 0:57:22.386881
seq 12 finished 1:01:34.281879
seq 13 finished 1:09:16.000767
seq 14 fi

C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:282: RuntimeWarning: divide by zero enc

seq 1 finished 0:01:27.688000
seq 2 finished 0:05:40.105999
seq 3 finished 0:06:47.287999
seq 4 finished 0:13:19.759999
seq 5 finished 0:19:50.063000
seq 6 finished 0:22:43.998999
seq 7 finished 0:32:48.616000
seq 8 finished 0:33:58.872000
seq 9 finished 0:35:45.018999
seq 10 finished 0:45:02.285999
seq 11 finished 0:46:10.747998
seq 12 finished 0:49:01.577000
seq 13 finished 0:54:47.252998
seq 14 finished 1:02:04.405999
seq 15 finished 1:10:46.746000
seq 16 finished 1:27:03.121001
seq 17 finished 1:49:11.740403

n_seq_warmup = 1
7
AR 0.15502037664449198 -0.4961132345320174 0.8506055724598771 0.8328858659984487
SMap 0.161708537654898 -0.4961132345320174 0.8499492870353798 0.8328858659984487
MSLRX 0.24761614929754527 -0.4961132345320174 0.8673110196289004 0.8328858659984487
MXLRXSolu3 0.24879672451531754 -0.4961132345320174 0.8685042658552592 0.8328858659984487
MSLR 0.15766653476019643 -0.4961132345320174 0.8758725613030249 0.8328858659984487
8
AR 0.15122451748902765 -0.5053997503781777

C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:282: RuntimeWarning: divide by zero enc

seq 1 finished 0:03:05.654805
seq 2 finished 0:09:16.578188
seq 3 finished 0:10:27.687047


C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_6096\466811895.py:282: RuntimeWarning: divide by zero enc

seq 4 finished 0:18:38.473445
seq 5 finished 0:26:34.287593
seq 6 finished 0:29:59.702628
seq 7 finished 0:42:14.174958
seq 8 finished 0:43:39.051288
seq 9 finished 0:46:04.217879
seq 10 finished 0:57:31.827218
seq 11 finished 0:58:55.400098
seq 12 finished 1:02:22.418297
seq 13 finished 1:09:12.385003
seq 14 finished 1:18:05.487103
seq 15 finished 1:29:13.220702
seq 16 finished 1:51:16.843303
seq 17 finished 2:18:30.269098

n_seq_warmup = 1
7
AR 0.04732577471702992 -0.4961132345320176 0.8439830559035857 0.8328858659984487
SMap 0.08797439919960504 -0.4961132345320176 0.8382256428614046 0.8328858659984487
MSLRX 0.0882626586951557 -0.4961132345320176 0.8697273432372771 0.8328858659984487
MXLRXSolu3 -0.23442759312119854 -0.4961132345320176 0.8456237694648291 0.8328858659984487
MSLR 0.09123989078996275 -0.4961132345320176 0.8692798759023925 0.8328858659984487
8
AR 0.042710193610104086 -0.5053997503781773 0.8385762385762385 0.8266153599486933
SMap 0.082954145618368 -0.5053997503781773 0.832

In [51]:

with open(r"res/res_vimsww_nowind_pred_0207_1.csv", "w") as fw:
    
    head = ["is_log10", "is_cross", "is_phase", "horizon_forecast", "reg_method", "n_seq_warmup", "len_warmup", "r2_naive", "rocauc_naive", "r2_ar", "rocauc_ar", "r2_smap", 
            "rocauc_smap", "r2_mslrx", "rocauc_mslrx", "tmp_r2_mslrxsolu3", "tmp_rocauc_mslrxsolu3", "tmp_r2_mslr", "tmp_rocauc_mslr", ]
    fw.write(",".join(head) + "\n")

    for tmp_setting in dict_setting_results:

        is_log, is_cross, is_phase, horizon_forecast, reg_method = tmp_setting
        dict_warmup_evaluation = dict_setting_results[tmp_setting]

        for n_seq_warmup in dict_warmup_evaluation:
            
            for cc in range(len(dict_warmup_evaluation[n_seq_warmup][0])):
                
                len_warmup = cc + 7
                tmp_r2_naive, tmp_rocauc_naive, tmp_r2_ar, tmp_rocauc_ar, tmp_r2_smap, tmp_rocauc_smap, tmp_r2_mslrx, tmp_rocauc_mslrx, tmp_r2_mslrxsolu3, tmp_rocauc_mslrxsolu3, tmp_r2_mslr, tmp_rocauc_mslr = dict_warmup_evaluation[n_seq_warmup][0][cc]
            
                line = [is_log, is_cross, is_phase, horizon_forecast, reg_method, n_seq_warmup, len_warmup, tmp_r2_naive, tmp_rocauc_naive, tmp_r2_ar, tmp_rocauc_ar, tmp_r2_smap, tmp_rocauc_smap, tmp_r2_mslrx, tmp_rocauc_mslrx, tmp_r2_mslrxsolu3, tmp_rocauc_mslrxsolu3, tmp_r2_mslr, tmp_rocauc_mslr]
                fw.write(",".join([str(i) for i in line]) + "\n")

    

## Predictions (WW) - save prediction results - feature importance

In [13]:

# # 1stmax
# data_WW_byday = data_WW[["date", "conductivity", "turbidity"]].resample("1D").max()
# data_WW_byday["temperature"] = data_WW["temperature"].resample("1D").mean()
# data_WW_byday["pH"] = data_WW["pH"].resample("1D").mean()
# data_WW_byday["ODO"] = data_WW["ODO"].resample("1D").min()
# data_WW_byday["salinity_max"] = data_WW["salinity"].resample("1D").max()
# data_WW_byday["salinity_min"] = data_WW["salinity"].resample("1D").min()
# data_WW_byday["log10_chlorophyll"] = data_WW["log10_chlorophyll"].resample("1D").max()
# data_WW_byday["chlorophyll"] = data_WW["chlorophyll"].resample("1D").max()

# 2ndmax
data_WW_byday = data_WW[["temperature", "pH"]].resample("1D").mean()
data_WW_byday["date"] = data_WW["date"].resample("1D").max()
data_WW_byday["conductivity"] = data_WW["conductivity"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
data_WW_byday["turbidity"] = data_WW["turbidity"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
data_WW_byday["ODO"] = data_WW["ODO"].resample("1D").apply(lambda x: x.nsmallest(2).iloc[-1] if len(x) > 0 else np.nan)
data_WW_byday["salinity_max"] = data_WW["salinity"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
data_WW_byday["salinity_min"] = data_WW["salinity"].resample("1D").apply(lambda x: x.nsmallest(2).iloc[-1] if len(x) > 0 else np.nan)
data_WW_byday["log10_chlorophyll"] = data_WW["log10_chlorophyll"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
data_WW_byday["chlorophyll"] = data_WW["chlorophyll"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)

# # 90-quantile
# data_WW_byday = data_WW[["temperature", "pH"]].resample("1D").mean()
# data_WW_byday["date"] = data_WW["date"].resample("1D").max()
# data_WW_byday["conductivity"] = data_WW["conductivity"].resample("1D").apply(lambda x: np.quantile(x, 0.9) if len(x) >= 2 else np.nan)
# data_WW_byday["turbidity"] = data_WW["turbidity"].resample("1D").apply(lambda x: np.quantile(x, 0.9) if len(x) >= 2 else np.nan)
# data_WW_byday["ODO"] = data_WW["ODO"].resample("1D").apply(lambda x: np.quantile(x, 0.1) if len(x) >= 2 else np.nan)
# data_WW_byday["salinity_max"] = data_WW["salinity"].resample("1D").apply(lambda x: np.quantile(x, 0.9) if len(x) >= 2 else np.nan)
# data_WW_byday["salinity_min"] = data_WW["salinity"].resample("1D").apply(lambda x: np.quantile(x, 0.1) if len(x) >= 2 else np.nan)
# data_WW_byday["log10_chlorophyll"] = data_WW["log10_chlorophyll"].resample("1D").apply(lambda x: np.quantile(x, 0.9) if len(x) >= 2 else np.nan)
# data_WW_byday["chlorophyll"] = data_WW["chlorophyll"].resample("1D").apply(lambda x: np.quantile(x, 0.9) if len(x) >= 2 else np.nan)


In [14]:
data_W_byday = data_W[["date"]].resample("1D").max()
#data_W_byday["airpressure"] = data_W["airpressure"].resample("1D").mean()
data_W_byday["wind"] = data_W["wind"].resample("1D").mean()

data_WW_byday = data_WW_byday.merge(data_W_byday, left_index=True, right_index=True).rename(columns = {"date_x": "date"}).drop(columns = ["date_y"])


In [15]:
data_WW_byday = data_WW_byday.dropna().resample("1D").max()

In [16]:
data_WW_byday

,temperature,pH,date,conductivity,turbidity,ODO,salinity_max,salinity_min,log10_chlorophyll,chlorophyll,wind
time_min,,,,,,,,,,,
2018-10-12,24.194146,7.902439,2018-10-12,46.755,44.39,6.32,30.38,28.48,1.285107,19.28,2.774167
2018-10-13,22.100917,7.863437,2018-10-13,46.714,26.13,5.89,30.38,28.42,1.426511,26.70,2.256528
2018-10-14,20.642937,7.871354,2018-10-14,46.585,16.85,6.08,30.31,28.71,1.306211,20.24,1.743472
2018-10-15,21.330021,7.870104,2018-10-15,46.370,15.62,6.33,30.15,28.73,1.272538,18.73,4.257870
2018-10-16,21.417677,7.843646,2018-10-16,46.481,15.39,5.69,30.23,28.61,1.211921,16.29,2.117500
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-20,5.802531,7.942708,2022-12-20,49.090,5.30,9.53,31.54,28.73,0.413300,2.59,1.510417
2022-12-21,5.474958,7.948021,2022-12-21,49.624,8.61,9.63,31.87,29.18,0.456366,2.86,2.171071
2022-12-22,5.977583,7.944375,2022-12-22,49.733,25.74,9.74,31.99,29.53,0.481443,3.03,4.545590


In [17]:
def _main(data_byday, 
          is_log = True, is_cross = True, is_phase = True, horizon_forecast = 1, thres_quantile = 0.95, 
          n_seq_warmup = 6, is_mslr = True, is_smap_cv = False, list_unused_feature = None,
          #reg_method = "LinearSVR", reg_method_func = LinearSVR, kargs_reg = {"random_state": 434},
          reg_method = "LinearRegression", reg_method_func = LinearRegression, kargs_reg = {},
         ):
    
    list_features = ["salinity_max", "salinity_min", "ODO", "turbidity", "pH", "temperature", "wind"]
    if list_unused_feature is not None:
        for tmp_feature in list_unused_feature:
            list_features.remove(tmp_feature)
            
    if is_log:
        array_obs = data_byday["log10_chlorophyll"].to_numpy()
    else:
        array_obs = data_byday["chlorophyll"].to_numpy()

    array_datetime = data_byday["date"].to_numpy()
    array_cross = data_byday[list_features].to_numpy()
    
    list_split_index = list(np.where(np.isnan(array_obs))[0])
    list_split_index.append(len(array_obs))
    list_split_index.insert(0, 0)
    list_subseq, list_subseq_cross,list_subseq_datetime = [], [], []
    for tmp_index in range(len(list_split_index) - 1):
        tmp_index_left = list_split_index[tmp_index] + 1
        tmp_index_right = list_split_index[tmp_index + 1]
        if tmp_index_right - tmp_index_left < 21:
            continue
        list_subseq.append(array_obs[tmp_index_left:tmp_index_right])
        list_subseq_datetime.append(array_datetime[tmp_index_left:tmp_index_right])
        list_subseq_cross.append(array_cross[tmp_index_left:tmp_index_right])
    
    list_p_AR = [2, 1, ]
    list_p_AR_cross = [1, ]
    list_p_AR.sort(reverse = True)
    max_p = max(max(list_p_AR), max(list_p_AR_cross) if len(list_p_AR_cross) != 0 else 0)

    list_X, list_Y, list_exog = [], [], []
    list_date = []
    for tmp_subseq, tmp_subseq_datetime, tmp_subseq_cross in zip(list_subseq, list_subseq_datetime, list_subseq_cross):
        X, Y, exog = [], [], []
        tmp_list_date = []
        for tmp_index_right in range(max_p, len(tmp_subseq) - horizon_forecast + 1):

            tmp_index_test = tmp_index_right - 1 + horizon_forecast # tmp_index_right == tmp_index_test if horizon_forecast = 1

            tmp_X = []
            if is_cross:
                for tmp_p in list_p_AR_cross:
                    tmp_X.extend(tmp_subseq_cross[tmp_index_right - tmp_p])
            for tmp_p in list_p_AR:
                tmp_X.append(tmp_subseq[tmp_index_right - tmp_p])
            X.append(tmp_X)
            Y.append([tmp_subseq[tmp_index_test]])

            tmp_date = tmp_subseq_datetime[tmp_index_test]
            tmp_list_date.append(tmp_date)
            
            tmp_t = pd.Timestamp(tmp_subseq_datetime[tmp_index_test]).toordinal() - pd.Timestamp(tmp_subseq_datetime[tmp_index_test]).replace(month = 1, day = 1).toordinal()
            if is_phase:
                tmp_exog = [np.cos(tmp_t / 365 * 2 * np.pi), np.sin(tmp_t / 365 * 2 * np.pi)]
            else:
                tmp_exog = [np.cos(tmp_t / 365 * 2 * np.pi)]
            exog.append(tmp_exog)

        list_X.append(X)
        list_Y.append(Y)
        list_exog.append(exog)
        list_date.append(tmp_list_date)
    
    print("Subseqs:")
    for tmp_index in range(len(list_subseq)):
        print("\t", tmp_index, pd.Timestamp(list_subseq_datetime[tmp_index][1]).date(), pd.Timestamp(list_subseq_datetime[tmp_index][-1]).date(), len(list_subseq[tmp_index]))
    print()
    
    # Prediction
    
    start_time = datetime.datetime.now()

    list_y_true, list_y_pred_naive = [], []
    list_y_pred_mslrx, list_y_pred_mslr = [], []
    list_y_pred_mslrxsolu3 = []
    list_y_pred_smap, list_y_pred_ar = [], []
    list_y_date = []
    list_y_seq, list_y_seq_index = [], []
    for cc in range(n_seq_warmup, len(list_X)):
        
        tmp_list_y_true, tmp_list_y_pred_naive = [], []
        tmp_list_y_pred_mslrx, tmp_list_y_pred_mslr = [], []
        tmp_list_y_pred_mslrxsolu3 = []
        tmp_list_y_pred_smap, tmp_list_y_pred_ar = [], []
        tmp_list_y_date = []
        tmp_list_y_seq, tmp_list_y_seq_index = [], []
        for tmp_index_test in range(max(5, horizon_forecast + 2), len(list_X[cc])):
            tmp_list_X, tmp_list_Y, tmp_list_exog = [], [], []
            tmp_list_X_test = []
            tmp_list_exog_additional = []
            for ii in range(cc):
                tmp_list_X.append(copy.deepcopy(list_X[ii]))
                tmp_list_Y.append(copy.deepcopy(list_Y[ii]))
                tmp_list_exog.append(copy.deepcopy(list_exog[ii]))
                tmp_list_X_test.append(copy.deepcopy(list_X[ii][-1]))
                if horizon_forecast != 1:
                    tmp_list_exog_additional.append(copy.deepcopy(list_exog[ii][(-horizon_forecast+1):]))
            tmp_list_X.append(copy.deepcopy(list_X[cc][:(tmp_index_test-horizon_forecast+1)]))
            tmp_list_Y.append(copy.deepcopy(list_Y[cc][:(tmp_index_test-horizon_forecast+1)]))
            tmp_list_exog.append(copy.deepcopy(list_exog[cc][:(tmp_index_test-horizon_forecast+1)]))
            tmp_list_X_test.append(copy.deepcopy(list_X[cc][tmp_index_test]))
            if horizon_forecast != 1:
                tmp_list_exog_additional.append(copy.deepcopy(list_exog[ii][(tmp_index_test-horizon_forecast+1):tmp_index_test]))
            
            tmp_y_true = list_Y[cc][tmp_index_test]
            tmp_y_pred_naive = list_X[cc][tmp_index_test][-1]
            
            if horizon_forecast ==  1:
                tmp_list_exog_additional = None
            
            if is_mslr:

                mslrx = MSLRX(n_components = 2, covariance_type="diag", n_iter = 10, is_logistic_regression_CV = False,
                              logistic_regression_C = 1e10, is_logistic_regression_standardized = False, reg_method = reg_method, kargs_reg = kargs_reg)
                tmp_pred = mslrx.fit_predict(tmp_list_X, tmp_list_Y, tmp_list_exog, tmp_list_X_test, is_multiple_sequence = True, exog_additional=tmp_list_exog_additional)
                tmp_y_pred = tmp_pred[-1]
                tmp_list_y_pred_mslrx.append(tmp_y_pred)

                mslrx = MSLRXSoluIII(n_components = 2, covariance_type="diag", n_iter = 10, reg_method = reg_method, kargs_reg = kargs_reg)
                tmp_pred = mslrx.fit_predict(tmp_list_X, tmp_list_Y, tmp_list_exog, tmp_list_X_test, is_multiple_sequence = True, exog_additional=tmp_list_exog_additional)
                tmp_y_pred = tmp_pred[-1]
                tmp_list_y_pred_mslrxsolu3.append(tmp_y_pred)

                mslr = MSLR(n_components = 2, covariance_type="diag", n_iter = 10, reg_method = reg_method, kargs_reg = kargs_reg)
                tmp_pred = mslr.fit_predict(tmp_list_X, tmp_list_Y, tmp_list_X_test, is_multiple_sequence = True, forecast_horizon=horizon_forecast)
                tmp_y_pred = tmp_pred[-1]
                tmp_list_y_pred_mslr.append(tmp_y_pred)
                
            else:
                tmp_list_y_pred_mslrx.append(0.)
                tmp_list_y_pred_mslrxsolu3.append(0.)
                tmp_list_y_pred_mslr.append(0.)
            
            lr = reg_method_func(**kargs_reg)
            lr.fit(np.concatenate(tmp_list_X), np.concatenate(tmp_list_Y).flatten())
            tmp_pred = lr.predict(tmp_list_X_test)
            tmp_y_pred = tmp_pred[-1]
            tmp_list_y_pred_ar.append(tmp_y_pred)
            
            if is_smap_cv:
                lr = SMapCV(thetas = (0.5, 1.0, 1.5, ), reg_method = reg_method, kargs_reg = kargs_reg)
            else:
                lr = SMap(theta = 0.5, reg_method = reg_method, kargs_reg = kargs_reg)
            lr.fit(np.concatenate(tmp_list_X), np.concatenate(tmp_list_Y).flatten())
            tmp_pred = lr.predict(tmp_list_X_test)
            tmp_y_pred = tmp_pred[-1]
            tmp_list_y_pred_smap.append(tmp_y_pred)

            tmp_list_y_true.append(tmp_y_true)
            tmp_list_y_pred_naive.append(tmp_y_pred_naive)
            tmp_list_y_date.append(list_date[cc][tmp_index_test])
            tmp_list_y_seq.append(cc)
            tmp_list_y_seq_index.append(tmp_index_test)
        
        list_y_date.append(tmp_list_y_date)
        list_y_seq.append(tmp_list_y_seq)
        list_y_seq_index.append(tmp_list_y_seq_index)
        if is_log:
            list_y_true.append(np.power(10, tmp_list_y_true))
            list_y_pred_naive.append(np.power(10, tmp_list_y_pred_naive))
            list_y_pred_mslrx.append(np.power(10, tmp_list_y_pred_mslrx))
            list_y_pred_mslr.append(np.power(10, tmp_list_y_pred_mslr))
            list_y_pred_mslrxsolu3.append(np.power(10, tmp_list_y_pred_mslrxsolu3))
            list_y_pred_ar.append(np.power(10, tmp_list_y_pred_ar))
            list_y_pred_smap.append(np.power(10, tmp_list_y_pred_smap))
        else:
            list_y_true.append(tmp_list_y_true)
            list_y_pred_naive.append(tmp_list_y_pred_naive)
            list_y_pred_mslrx.append(tmp_list_y_pred_mslrx)
            list_y_pred_mslr.append(tmp_list_y_pred_mslr)
            list_y_pred_mslrxsolu3.append(tmp_list_y_pred_mslrxsolu3)
            list_y_pred_ar.append(tmp_list_y_pred_ar)
            list_y_pred_smap.append(tmp_list_y_pred_smap)
        
        print("seq % s finished" % cc, str(datetime.datetime.now() - start_time))
        
    list_y_date, list_y_true, list_y_pred_naive = np.concatenate(list_y_date).flatten(), np.concatenate(list_y_true).flatten(), np.concatenate(list_y_pred_naive).flatten()
    list_y_pred_mslrx, list_y_pred_mslr, list_y_pred_mslrxsolu3, list_y_pred_ar, list_y_pred_smap = np.concatenate(list_y_pred_mslrx).flatten(), np.concatenate(list_y_pred_mslr).flatten(), np.concatenate(list_y_pred_mslrxsolu3).flatten(), np.concatenate(list_y_pred_ar).flatten(), np.concatenate(list_y_pred_smap).flatten() 
    list_y_seq, list_y_seq_index = np.concatenate(list_y_seq).flatten(), np.concatenate(list_y_seq_index).flatten()
    
    dict_method_pred = {"date": list_y_date, 
                        "true": list_y_true, 
                        "pred_naive": list_y_pred_naive, 
                        "pred_mslrx": list_y_pred_mslrx,
                        "pred_mslr": list_y_pred_mslr,
                        "pred_mslrxsolu3": list_y_pred_mslrxsolu3, 
                        "pred_ar": list_y_pred_ar,
                        "pred_smap": list_y_pred_smap,
                        "seq": list_y_seq,
                        "seq_index": list_y_seq_index,
                       }
                
    return pd.DataFrame(dict_method_pred)


In [18]:

is_log = True
is_phase = True

dict_setting_results = dict()

for is_cross in [True]:
    for horizon_forecast in [1, 3, 7]:
        for feature_removed in [None, "salinity_max", "salinity_min", "ODO", "turbidity", "pH", "temperature", "wind"]:
        # for feature_removed in [None, ]:
                
                tmp_setting = (is_log, is_cross, is_phase, horizon_forecast, "LinearRegression", )
                
                print()
                print("================================================")
                print("setting =", tmp_setting)
                print("removed feature =", feature_removed)
                print("================================================")
                
                tmp_res_df = _main(data_WW_byday, n_seq_warmup=1, 
                                  #reg_method = "LinearSVR", reg_method_func = LinearSVR, kargs_reg = {"random_state": 434},
                                  reg_method = "LinearRegression", reg_method_func = LinearRegression, kargs_reg = {},
                                  is_log = is_log, is_cross = is_cross, is_phase = is_phase, horizon_forecast = horizon_forecast, is_mslr = True,
                                   list_unused_feature = [feature_removed] if feature_removed is not None else None,
                                  )
                
                dict_setting_results[tmp_setting] = tmp_res_df
                
                tail_log = "_log" if is_log else ""
                tail_cross = "_X" if is_cross else ""
                tail_horizon = "% sD" % horizon_forecast
                tail_method = "LR"
                
                tail_removed_features = "EX" + feature_removed.replace("_", "").lower() if feature_removed is not None else "allX" 
                
                tail_max = "2ndmax"
                # tail_max = "1stmax"
                #tail_max = "90qmax"
                # tail_max = "smooth"
                
                fname_write = "vimsWW_% s_pred_% s_% s_% s% s% s.csv" % (tail_max, tail_removed_features, tail_method, tail_horizon, tail_cross, tail_log, )
                tmp_res_df.to_csv(os.path.join("res", "vimsWW_pred_truehorizon", fname_write), index = False)



setting = (True, True, True, 1, 'LinearRegression')
removed feature = None
Subseqs:
	 0 2018-11-01 2019-01-22 84
	 1 2019-02-02 2019-03-24 52
	 2 2019-03-27 2019-06-24 91
	 3 2019-06-27 2019-07-25 30
	 4 2019-08-02 2019-11-16 108
	 5 2019-11-20 2020-02-12 86
	 6 2020-02-15 2020-03-27 43
	 7 2020-03-31 2020-07-16 109
	 8 2020-09-04 2020-09-25 23
	 9 2021-02-09 2021-03-07 28
	 10 2021-04-01 2021-06-28 90
	 11 2021-07-01 2021-07-21 22
	 12 2021-07-29 2021-08-31 35
	 13 2021-09-03 2021-10-27 56
	 14 2021-10-30 2022-01-01 65
	 15 2022-01-15 2022-03-26 72
	 16 2022-03-29 2022-07-23 118
	 17 2022-07-28 2022-12-09 136

seq 1 finished 0:02:26.971972
seq 2 finished 0:07:16.576142
seq 3 finished 0:09:02.020136



KeyboardInterrupt



## Predictions (W)

In [24]:
data_W_byday = data_W[["date"]].resample("1D").max()
data_W_byday["temperature"] = data_W["temperature"].resample("1D").mean()
data_W_byday["airpressure"] = data_W["airpressure"].resample("1D").mean()
data_W_byday["wind"] = data_W["wind"].resample("1D").mean()
data_W_byday["waterlevel"] = data_W["waterlevel"].resample("1D").mean()
data_W_byday["ODO"] = data_W["ODO"].resample("1D").min()
data_W_byday["salinity_max"] = data_W["salinity"].resample("1D").max()
data_W_byday["salinity_min"] = data_W["salinity"].resample("1D").min()
data_W_byday["log10_chlorophyll"] = data_W["log10_chlorophyll"].resample("1D").max()
data_W_byday["chlorophyll"] = data_W["chlorophyll"].resample("1D").max()

# data_W_byday = data_W[["temperature", "pH"]].resample("1D").mean()
# data_W_byday["date"] = data_W["date"].resample("1D").max()
# data_W_byday["conductivity"] = data_W["conductivity"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_W_byday["turbidity"] = data_W["turbidity"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_W_byday["ODO"] = data_W["ODO"].resample("1D").apply(lambda x: x.nsmallest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_W_byday["salinity_max"] = data_W["salinity"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_W_byday["salinity_min"] = data_W["salinity"].resample("1D").apply(lambda x: x.nsmallest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_W_byday["log10_chlorophyll"] = data_W["log10_chlorophyll"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)
# data_W_byday["chlorophyll"] = data_W["chlorophyll"].resample("1D").apply(lambda x: x.nlargest(2).iloc[-1] if len(x) > 0 else np.nan)


In [25]:
def _main(data_WW_byday, is_log = True, is_cross = True, is_phase = True, horizon_forecast = 1, thres_quantile = 0.95, n_seq_warmup = 6,
          reg_method = "LinearSVR", reg_method_func = LinearSVR, kargs_reg = {"random_state": 434},
          #reg_method = "LinearRegression", reg_method_func = LinearRegression, kargs_reg = {},
         ):
    
    # Data Preparation
    if is_log:
        array_obs = data_WW_byday["log10_chlorophyll"].to_numpy()
    else:
        array_obs = data_WW_byday["chlorophyll"].to_numpy()

    array_datetime = data_WW_byday["date"].to_numpy()
    #array_cross = data_WW_byday[["salinity_max", "salinity_min", "ODO", "turbidity", "pH", "temperature"]].to_numpy()
    #array_cross = data_WW_byday[["temperature", "airpressure", "wind", "waterlevel", "ODO", "salinity_max", "salinity_min"]].to_numpy()
    array_cross = data_WW_byday[["temperature", "ODO", "salinity_max", "salinity_min"]].to_numpy()
    
    list_split_index = list(np.where(np.isnan(array_obs))[0])
    list_split_index.append(len(array_obs))
    list_split_index.insert(0, 0)
    list_subseq, list_subseq_cross,list_subseq_datetime = [], [], []
    for tmp_index in range(len(list_split_index) - 1):
        tmp_index_left = list_split_index[tmp_index] + 1
        tmp_index_right = list_split_index[tmp_index + 1]
        if tmp_index_right - tmp_index_left < 21:
            continue
        list_subseq.append(array_obs[tmp_index_left:tmp_index_right])
        list_subseq_datetime.append(array_datetime[tmp_index_left:tmp_index_right])
        list_subseq_cross.append(array_cross[tmp_index_left:tmp_index_right])
    
    list_p_AR = [2, 1, ]
    list_p_AR_cross = [1, ]
    list_p_AR.sort(reverse = True)
    max_p = max(max(list_p_AR), max(list_p_AR_cross) if len(list_p_AR_cross) != 0 else 0)

    list_X, list_Y, list_exog = [], [], []
    list_date = []
    for tmp_subseq, tmp_subseq_datetime, tmp_subseq_cross in zip(list_subseq, list_subseq_datetime, list_subseq_cross):
        X, Y, exog = [], [], []
        tmp_list_date = []
        for tmp_index_right in range(max_p, len(tmp_subseq) - horizon_forecast + 1):

            tmp_index_test = tmp_index_right - 1 + horizon_forecast # tmp_index_right == tmp_index_test if horizon_forecast = 1

            tmp_X = []
            if is_cross:
                for tmp_p in list_p_AR_cross:
                    tmp_X.extend(tmp_subseq_cross[tmp_index_right - tmp_p])
            for tmp_p in list_p_AR:
                tmp_X.append(tmp_subseq[tmp_index_right - tmp_p])
            X.append(tmp_X)
            Y.append([tmp_subseq[tmp_index_test]])

            tmp_date= tmp_subseq_datetime[tmp_index_test]
            tmp_list_date.append(tmp_date)
            
            tmp_t = pd.Timestamp(tmp_subseq_datetime[tmp_index_test]).toordinal() - pd.Timestamp(tmp_subseq_datetime[tmp_index_test]).replace(month = 1, day = 1).toordinal()
            if is_phase:
                tmp_exog = [np.cos(tmp_t / 365 * 2 * np.pi), np.sin(tmp_t / 365 * 2 * np.pi)]
            else:
                tmp_exog = [np.cos(tmp_t / 365 * 2 * np.pi)]
            exog.append(tmp_exog)

        list_X.append(X)
        list_Y.append(Y)
        list_exog.append(exog)
        list_date.append(tmp_list_date)
    
    print("Subseqs:")
    for tmp_index in range(len(list_subseq)):
        print("\t", tmp_index, pd.Timestamp(list_subseq_datetime[tmp_index][1]).date(), pd.Timestamp(list_subseq_datetime[tmp_index][-1]).date(), len(list_subseq[tmp_index]))
    print()
    
    # Prediction
    
    start_time = datetime.datetime.now()

    list_y_true, list_y_pred_naive = [], []
    list_y_pred_mslrx, list_y_pred_mslr = [], []
    list_y_pred_mslrxsolu3 = []
    list_y_date = []
    for cc in range(n_seq_warmup, len(list_X)):
        tmp_list_y_true, tmp_list_y_pred_naive = [], []
        tmp_list_y_pred_mslrx, tmp_list_y_pred_mslr = [], []
        tmp_list_y_pred_mslrxsolu3 = []
        tmp_list_y_date = []
        for tmp_index_test in range(5, len(list_X[cc])):
            tmp_list_X, tmp_list_Y, tmp_list_exog = [], [], []
            tmp_list_X_test = []
            for ii in range(cc):
                tmp_list_X.append(copy.deepcopy(list_X[ii]))
                tmp_list_Y.append(copy.deepcopy(list_Y[ii]))
                tmp_list_exog.append(copy.deepcopy(list_exog[ii]))
                tmp_list_X_test.append(copy.deepcopy(list_X[ii][-1]))
            tmp_list_X.append(copy.deepcopy(list_X[cc][:tmp_index_test]))
            tmp_list_Y.append(copy.deepcopy(list_Y[cc][:tmp_index_test]))
            tmp_list_exog.append(copy.deepcopy(list_exog[cc][:tmp_index_test]))
            tmp_list_X_test.append(copy.deepcopy(list_X[cc][tmp_index_test]))

            mslrx = MSLRX(n_components = 2, covariance_type="diag", n_iter = 10, is_logistic_regression_CV = False,
                          logistic_regression_C = 1e10, is_logistic_regression_standardized = False, reg_method = reg_method, kargs_reg = kargs_reg)
            tmp_pred = mslrx.fit_predict(tmp_list_X, tmp_list_Y, tmp_list_exog, tmp_list_X_test, is_multiple_sequence = True)
            tmp_y_pred = tmp_pred[-1]
            tmp_list_y_pred_mslrx.append(tmp_y_pred)

            mslrx = MSLRXSoluIII(n_components = 2, covariance_type="diag", n_iter = 10, reg_method = reg_method, kargs_reg = kargs_reg)
            tmp_pred = mslrx.fit_predict(tmp_list_X, tmp_list_Y, tmp_list_exog, tmp_list_X_test, is_multiple_sequence = True)
            tmp_y_pred = tmp_pred[-1]
            tmp_list_y_pred_mslrxsolu3.append(tmp_y_pred)

            mslr = MSLR(n_components = 2, covariance_type="diag", n_iter = 10, reg_method = reg_method, kargs_reg = kargs_reg)
            tmp_pred = mslr.fit_predict(tmp_list_X, tmp_list_Y, tmp_list_X_test, is_multiple_sequence = True)
            tmp_y_pred = tmp_pred[-1]
            tmp_list_y_pred_mslr.append(tmp_y_pred)

            tmp_y_true = list_Y[cc][tmp_index_test]
            #tmp_y_pred_naive = list_Y[cc][tmp_index_test - 1]
            tmp_y_pred_naive = list_X[cc][tmp_index_test][-1]
            tmp_list_y_true.append(tmp_y_true)
            tmp_list_y_pred_naive.append(tmp_y_pred_naive)
            tmp_list_y_date.append(list_date[cc][tmp_index_test])

        list_y_true.append(tmp_list_y_true)
        list_y_pred_naive.append(tmp_list_y_pred_naive)
        list_y_pred_mslrx.append(tmp_list_y_pred_mslrx)
        list_y_pred_mslr.append(tmp_list_y_pred_mslr)
        list_y_pred_mslrxsolu3.append(tmp_list_y_pred_mslrxsolu3)
        list_y_date.append(tmp_list_y_date)

        print("seq % s finished" % cc, str(datetime.datetime.now() - start_time))

        #if cc > 5: break
        
    if is_log:
        thres = np.nanquantile(np.power(10, array_obs), thres_quantile)
    else:
        thres = np.nanquantile(array_obs, thres_quantile)

    list_p_AR = [2, 1, ]
    list_p_AR_cross = [1, ]
    list_p_AR.sort(reverse = True)
    
    if is_cross:
        max_p = max(max(list_p_AR), max(list_p_AR_cross) if len(list_p_AR_cross) != 0 else 0)
    else:
        max_p = max(list_p_AR)
    
    dict_warmup_evaluation = dict()
    for n_seq_exclude_valid in range(n_seq_warmup, len(list_subseq) - 2):
        
        print()
        print("n_seq_warmup =", n_seq_exclude_valid)
        
        list_res_logitcos_cross = []
        list_res_pred = []
        for start_index in range(7, 20):

            print(start_index)

            X, Y = [], []
            list_index_test_set = []
            list_date_test = []
            for cc, (tmp_subseq, tmp_subseq_datetime, tmp_subseq_cross) in enumerate(zip(list_subseq, list_subseq_datetime, list_subseq_cross)):
                for tmp_index_right in range(max_p, len(tmp_subseq) - horizon_forecast + 1):
                    tmp_index_test = tmp_index_right - 1 + horizon_forecast # tmp_index_right == tmp_index_test if horizon_forecast = 1
                    tmp_X = []
                    if is_cross:
                        for tmp_p in list_p_AR_cross:
                            tmp_X.extend(tmp_subseq_cross[tmp_index_right - tmp_p])
                    for tmp_p in list_p_AR:
                        tmp_X.append(tmp_subseq[tmp_index_right - tmp_p])
                    X.append(tmp_X)
                    Y.append([tmp_subseq[tmp_index_test]])

                    if cc >= n_seq_exclude_valid and tmp_index_right >= start_index:
                        list_index_test_set.append(len(Y) - 1)
                        list_date_test.append(tmp_subseq_datetime[tmp_index_test])

            y_true, y_pred = [], []
            y_pred_naive = []
            for tmp_index in list_index_test_set:
                lr = reg_method_func(**kargs_reg)
                #lr = SMap(theta = 0.5, reg_method = reg_method, kargs_reg = kargs_reg)
                #lr = SMap(theta = 0.5)
                lr.fit(X[:tmp_index], Y[:tmp_index])
                tmp_y_pred = lr.predict([X[tmp_index]])[0]
                tmp_y_true = Y[tmp_index]
                tmp_y_pred_naive = X[tmp_index][-1]
                y_true.append(tmp_y_true)
                y_pred.append(tmp_y_pred)
                y_pred_naive.append(tmp_y_pred_naive)

            if is_log:
                y_true = np.power(10, y_true)
                y_pred_naive = np.power(10, y_pred_naive)
                y_pred = np.power(10, y_pred)

            tmp_r2_ar, tmp_r2_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive)
            tmp_rocauc_ar, tmp_rocauc_naive = roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
            y_pred_ar = y_pred
            print("AR", tmp_r2_ar, tmp_r2_naive, tmp_rocauc_ar, tmp_rocauc_naive)
            
            y_true, y_pred = [], []
            y_pred_naive = []
            for tmp_index in list_index_test_set:
                #lr = reg_method_func(**kargs_reg)
                lr = SMap(theta = 0.5, reg_method = reg_method, kargs_reg = kargs_reg)
                #lr = SMap(theta = 0.5)
                lr.fit(X[:tmp_index], Y[:tmp_index])
                tmp_y_pred = lr.predict([X[tmp_index]])[0]
                tmp_y_true = Y[tmp_index]
                tmp_y_pred_naive = X[tmp_index][-1]
                y_true.append(tmp_y_true)
                y_pred.append(tmp_y_pred)
                y_pred_naive.append(tmp_y_pred_naive)

            if is_log:
                y_true = np.power(10, y_true)
                y_pred_naive = np.power(10, y_pred_naive)
                y_pred = np.power(10, y_pred)

            tmp_r2_smap, tmp_r2_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive)
            tmp_rocauc_smap, tmp_rocauc_naive = roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
            y_pred_smap = y_pred
            print("SMap", tmp_r2_smap, tmp_r2_naive, tmp_rocauc_smap, tmp_rocauc_naive)

            y_true, y_pred = [], []
            y_pred_naive = []
            for cc in range(n_seq_exclude_valid - n_seq_warmup, len(list_y_true)):
                for tmp_index_test in range(start_index - 5 - max_p, len(list_y_true[cc])):
                    y_true.append(list_y_true[cc][tmp_index_test])
                    y_pred.append(list_y_pred_mslrx[cc][tmp_index_test])
                    y_pred_naive.append(list_y_pred_naive[cc][tmp_index_test])

            if is_log:
                y_true = np.power(10, y_true)
                y_pred_naive = np.power(10, y_pred_naive)
                y_pred = np.power(10, y_pred)

            tmp_r2_mslrx, tmp_r2_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive)
            tmp_rocauc_mslrx, tmp_rocauc_naive = roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
            y_pred_mslrx = y_pred
            print("MSLRX", tmp_r2_mslrx, tmp_r2_naive, tmp_rocauc_mslrx, tmp_rocauc_naive)

            y_true, y_pred = [], []
            y_pred_naive = []
            for cc in range(n_seq_exclude_valid - n_seq_warmup, len(list_y_true)):
                for tmp_index_test in range(start_index - 5 - max_p, len(list_y_true[cc])):
                    y_true.append(list_y_true[cc][tmp_index_test])
                    y_pred.append(list_y_pred_mslrxsolu3[cc][tmp_index_test])
                    y_pred_naive.append(list_y_pred_naive[cc][tmp_index_test])

            if is_log:
                y_true = np.power(10, y_true)
                y_pred_naive = np.power(10, y_pred_naive)
                y_pred = np.power(10, y_pred)

            tmp_r2_mslrxsolu3, tmp_r2_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive)
            tmp_rocauc_mslrxsolu3, tmp_rocauc_naive = roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
            y_pred_mslrxsolu3 = y_pred
            print("MXLRXSolu3", tmp_r2_mslrxsolu3, tmp_r2_naive, tmp_rocauc_mslrxsolu3, tmp_rocauc_naive)

            y_true, y_pred = [], []
            y_pred_naive = []
            for cc in range(n_seq_exclude_valid - n_seq_warmup, len(list_y_true)):
                for tmp_index_test in range(start_index - 5 - max_p, len(list_y_true[cc])):
                    y_true.append(list_y_true[cc][tmp_index_test])
                    y_pred.append(list_y_pred_mslr[cc][tmp_index_test])
                    y_pred_naive.append(list_y_pred_naive[cc][tmp_index_test])

            if is_log:
                y_true = np.power(10, y_true)
                y_pred_naive = np.power(10, y_pred_naive)
                y_pred = np.power(10, y_pred)

            tmp_r2_mslr, tmp_r2_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive)
            tmp_rocauc_mslr, tmp_rocauc_naive = roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
            y_pred_mslr = y_pred
            print("MSLR", tmp_r2_mslr, tmp_r2_naive, tmp_rocauc_mslr, tmp_rocauc_naive)

            list_res_logitcos_cross.append((tmp_r2_naive, tmp_rocauc_naive,
                                            tmp_r2_ar, tmp_rocauc_ar, 
                                            tmp_r2_smap, tmp_rocauc_smap,
                                            tmp_r2_mslrx, tmp_rocauc_mslrx, 
                                            tmp_r2_mslrxsolu3, tmp_rocauc_mslrxsolu3, 
                                            tmp_r2_mslr, tmp_rocauc_mslr,
                                           ))
            
            list_res_pred.append((list_date_test, y_true, y_pred_naive, y_pred_ar, y_pred_smap, y_pred_mslrx, y_pred_mslrxsolu3, y_pred_mslr))
            
        dict_warmup_evaluation[n_seq_exclude_valid] = (list_res_logitcos_cross, list_res_pred)
                
    return dict_warmup_evaluation


In [ ]:

dict_setting_results = dict()
for is_log in [False]:
    for is_cross in [True, False]:
        for is_phase in [True]:
            for horizon_forecast in [1, 3, 7]:
                
                tmp_setting = (is_log, is_cross, is_phase, horizon_forecast, "LinearRegression", )
                
                print()
                print("================================================")
                print("setting =", tmp_setting)
                print("================================================")
                
                dict_setting_results[tmp_setting] = _main(data_W_byday, n_seq_warmup=1, 
                                                          thres_quantile = 0.9, 
                                                          #reg_method = "LinearSVR", reg_method_func = LinearSVR, kargs_reg = {"random_state": 434},
                                                          reg_method = "LinearRegression", reg_method_func = LinearRegression, kargs_reg = {},
                                                          is_log = is_log, is_cross = is_cross, is_phase = is_phase, horizon_forecast = horizon_forecast,)
                 
                



setting = (False, True, True, 1, 'LinearRegression')
Subseqs:
	 0 2016-04-19 2016-05-10 23
	 1 2016-06-07 2016-06-30 25
	 2 2016-07-19 2016-08-13 27
	 3 2016-08-31 2016-11-23 86
	 4 2017-04-06 2017-05-11 37
	 5 2017-06-16 2017-07-13 29
	 6 2017-07-19 2017-08-08 22
	 7 2017-09-13 2017-10-19 38
	 8 2017-11-01 2017-12-10 41
	 9 2018-01-26 2018-02-22 29
	 10 2018-04-10 2018-06-15 68
	 11 2019-02-05 2019-03-12 37
	 12 2019-03-21 2019-04-17 29
	 13 2019-04-25 2019-05-21 28
	 14 2019-05-24 2019-07-25 64
	 15 2019-08-01 2019-09-02 34
	 16 2019-09-06 2019-09-27 23
	 17 2020-04-15 2020-05-04 21
	 18 2020-12-12 2021-01-27 48
	 19 2021-03-26 2021-08-05 134
	 20 2021-08-13 2021-12-01 112
	 21 2021-12-16 2022-10-23 313
	 22 2022-10-26 2022-12-31 68



C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero

seq 1 finished 0:00:22.085839


C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero

seq 2 finished 0:00:56.683312


...

C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero

seq 2 finished 0:00:48.573955


C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero

seq 3 finished 0:03:59.250561
seq 4 finished 0:05:29.928284
seq 5 finished 0:06:41.668765
seq 6 finished 0:07:31.716177
seq 7 finished 0:09:32.645386
seq 8 finished 0:11:59.712696
seq 9 finished 0:13:34.011474
seq 10 finished 0:18:34.363268
seq 11 finished 0:21:10.603760
seq 12 finished 0:23:06.243623
seq 13 finished 0:25:01.327992
seq 14 finished 0:31:02.629329
seq 15 finished 0:33:55.625354
seq 16 finished 0:35:34.992133
seq 17 finished 0:37:01.602829
seq 18 finished 0:41:53.496329
seq 19 finished 0:58:43.881697
seq 20 finished 1:14:03.401921
seq 21 finished 2:06:41.081108



KeyboardInterrupt



In [27]:

with open(r"res/res_vimsw_pred_0205_3.csv", "w") as fw:
    
    head = ["is_log10", "is_cross", "is_phase", "horizon_forecast", "reg_method", "n_seq_warmup", "len_warmup", "r2_naive", "rocauc_naive", "r2_ar", "rocauc_ar", "r2_smap", 
            "rocauc_smap", "r2_mslrx", "rocauc_mslrx", "tmp_r2_mslrxsolu3", "tmp_rocauc_mslrxsolu3", "tmp_r2_mslr", "tmp_rocauc_mslr", ]
    fw.write(",".join(head) + "\n")

    for tmp_setting in dict_setting_results:

        is_log, is_cross, is_phase, horizon_forecast, reg_method = tmp_setting
        dict_warmup_evaluation = dict_setting_results[tmp_setting]

        for n_seq_warmup in dict_warmup_evaluation:
            
            for cc in range(len(dict_warmup_evaluation[n_seq_warmup][0])):
                
                len_warmup = cc + 7
                tmp_r2_naive, tmp_rocauc_naive, tmp_r2_ar, tmp_rocauc_ar, tmp_r2_smap, tmp_rocauc_smap, tmp_r2_mslrx, tmp_rocauc_mslrx, tmp_r2_mslrxsolu3, tmp_rocauc_mslrxsolu3, tmp_r2_mslr, tmp_rocauc_mslr = dict_warmup_evaluation[n_seq_warmup][0][cc]
            
                line = [is_log, is_cross, is_phase, horizon_forecast, reg_method, n_seq_warmup, len_warmup, tmp_r2_naive, tmp_rocauc_naive, tmp_r2_ar, tmp_rocauc_ar, tmp_r2_smap, tmp_rocauc_smap, tmp_r2_mslrx, tmp_rocauc_mslrx, tmp_r2_mslrxsolu3, tmp_rocauc_mslrxsolu3, tmp_r2_mslr, tmp_rocauc_mslr]
                fw.write(",".join([str(i) for i in line]) + "\n")

    

In [29]:

dict_setting_results = dict()
for is_log in [False]:
    for is_cross in [True, False]:
        for is_phase in [True]:
            for horizon_forecast in [1, 3, 7]:
                
                #tmp_setting = (is_log, is_cross, is_phase, horizon_forecast, "LinearRegression", )
                tmp_setting = (is_log, is_cross, is_phase, horizon_forecast, "LinearSVR", )
                
                print()
                print("================================================")
                print("setting =", tmp_setting)
                print("================================================")
                
                dict_setting_results[tmp_setting] = _main(data_W_byday, n_seq_warmup=1, 
                                                          thres_quantile = 0.9, 
                                                          reg_method = "LinearSVR", reg_method_func = LinearSVR, kargs_reg = {"random_state": 434},
                                                          #reg_method = "LinearRegression", reg_method_func = LinearRegression, kargs_reg = {},
                                                          is_log = is_log, is_cross = is_cross, is_phase = is_phase, horizon_forecast = horizon_forecast,)
                 
                



setting = (False, True, True, 1, 'LinearSVR')
Subseqs:
	 0 2016-04-19 2016-05-10 23
	 1 2016-06-07 2016-06-30 25
	 2 2016-07-19 2016-08-13 27
	 3 2016-08-31 2016-11-23 86
	 4 2017-04-06 2017-05-11 37
	 5 2017-06-16 2017-07-13 29
	 6 2017-07-19 2017-08-08 22
	 7 2017-09-13 2017-10-19 38
	 8 2017-11-01 2017-12-10 41
	 9 2018-01-26 2018-02-22 29
	 10 2018-04-10 2018-06-15 68
	 11 2019-02-05 2019-03-12 37
	 12 2019-03-21 2019-04-17 29
	 13 2019-04-25 2019-05-21 28
	 14 2019-05-24 2019-07-25 64
	 15 2019-08-01 2019-09-02 34
	 16 2019-09-06 2019-09-27 23
	 17 2020-04-15 2020-05-04 21
	 18 2020-12-12 2021-01-27 48
	 19 2021-03-26 2021-08-05 134
	 20 2021-08-13 2021-12-01 112
	 21 2021-12-16 2022-10-23 313
	 22 2022-10-26 2022-12-31 68

seq 1 finished 0:00:25.823002


C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \


seq 2 finished 0:00:51.964000
seq 3 finished 0:03:31.842803
seq 4 finished 0:04:54.306539
seq 5 finished 0:06:01.023541
seq 6 finished 0:06:52.010571
seq 7 finished 0:08:43.520541
seq 8 finished 0:11:01.106541
seq 9 finished 0:12:32.835542
seq 10 finished 0:17:16.293541
seq 11 finished 0:19:49.445541
seq 12 finished 0:21:48.426544
seq 13 finished 0:23:46.915541
seq 14 finished 0:29:57.448544
seq 15 finished 0:33:14.948542
seq 16 finished 0:35:17.614541
seq 17 finished 0:37:08.892541
seq 18 finished 0:42:46.580542
seq 19 finished 1:03:09.510542
seq 20 finished 1:22:27.461543
seq 21 finished 2:25:23.061541
seq 22 finished 2:39:23.973541

n_seq_warmup = 1
7
AR 0.7581439905384861 0.7776165948741907 0.947247237003684 0.9535577841451767
SMap 0.7747552612514498 0.7776165948741907 0.9497032337290217 0.9535577841451767
MSLRX 0.765116908321211 0.7776165948741907 0.9495412061672807 0.9535577841451767
MXLRXSolu3 0.7629118521190439 0.7776165948741907 0.9477077363896849 0.9535577841451767
MSLR 0.766

C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero

seq 1 finished 0:00:18.295999


...

seq 2 finished 0:00:29.501349


C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero

seq 3 finished 0:02:53.149590


C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:269: RuntimeWarning: divide by zero encountered in log
  tmp_log_prob.append(logsumexp([np.log(list_cur_list_trans_mat[index_X][tt][ii, j]) + list_cur_mat_log_b[index_X][tt + 1, j] + list_cur_log_backward_prob[index_X][tt + 1][j] for j in range(K)]))
C:\Users\noah_\AppData\Local\Temp\ipykernel_15788\466811895.py:282: RuntimeWarning: divide by zero encountered in log
  list_tmp_array3d_log_epsilon[index_X][tt, ii, jj] = list_cur_log_forward_prob[index_X][tt, ii] + np.log(list_cur_list_trans_mat[index_X][tt][ii, jj]) + \


seq 4 finished 0:03:57.107588
seq 5 finished 0:04:43.869970
seq 6 finished 0:05:11.273159
seq 7 finished 0:06:34.079446
seq 8 finished 0:08:17.828679
seq 9 finished 0:09:17.798452
seq 10 finished 0:13:06.164795
seq 11 finished 0:14:55.224388
seq 12 finished 0:16:18.033379
seq 13 finished 0:17:35.987265
seq 14 finished 0:22:22.037448
seq 15 finished 0:24:27.154374
seq 16 finished 0:25:28.406494
seq 17 finished 0:26:18.573828
seq 18 finished 0:30:10.834886
seq 19 finished 0:45:10.894912
seq 20 finished 0:59:23.119485
seq 21 finished 1:51:49.088194
seq 22 finished 2:01:51.524489

n_seq_warmup = 1
7
AR 0.1279646353509487 -0.10253890850258252 0.7603743327255552 0.758781661211309
SMap 0.1759223905334848 -0.10253890850258252 0.7596054568221261 0.758781661211309
MSLRX 0.361882488669389 -0.10253890850258252 0.8306056545330728 0.758781661211309
MXLRXSolu3 0.3153697502778374 -0.10253890850258252 0.7965993717185474 0.758781661211309
MSLR 0.3621584480180915 -0.10253890850258252 0.8151512488741459 0

In [30]:

with open(r"res/res_vimsw_pred_0205_1.csv", "w") as fw:
    
    head = ["is_log10", "is_cross", "is_phase", "horizon_forecast", "reg_method", "n_seq_warmup", "len_warmup", "r2_naive", "rocauc_naive", "r2_ar", "rocauc_ar", "r2_smap", 
            "rocauc_smap", "r2_mslrx", "rocauc_mslrx", "tmp_r2_mslrxsolu3", "tmp_rocauc_mslrxsolu3", "tmp_r2_mslr", "tmp_rocauc_mslr", ]
    fw.write(",".join(head) + "\n")

    for tmp_setting in dict_setting_results:

        is_log, is_cross, is_phase, horizon_forecast, reg_method = tmp_setting
        dict_warmup_evaluation = dict_setting_results[tmp_setting]

        for n_seq_warmup in dict_warmup_evaluation:
            
            for cc in range(len(dict_warmup_evaluation[n_seq_warmup][0])):
                
                len_warmup = cc + 7
                tmp_r2_naive, tmp_rocauc_naive, tmp_r2_ar, tmp_rocauc_ar, tmp_r2_smap, tmp_rocauc_smap, tmp_r2_mslrx, tmp_rocauc_mslrx, tmp_r2_mslrxsolu3, tmp_rocauc_mslrxsolu3, tmp_r2_mslr, tmp_rocauc_mslr = dict_warmup_evaluation[n_seq_warmup][0][cc]
            
                line = [is_log, is_cross, is_phase, horizon_forecast, reg_method, n_seq_warmup, len_warmup, tmp_r2_naive, tmp_rocauc_naive, tmp_r2_ar, tmp_rocauc_ar, tmp_r2_smap, tmp_rocauc_smap, tmp_r2_mslrx, tmp_rocauc_mslrx, tmp_r2_mslrxsolu3, tmp_rocauc_mslrxsolu3, tmp_r2_mslr, tmp_rocauc_mslr]
                fw.write(",".join([str(i) for i in line]) + "\n")

    

## End of Notebook